### This notebook includes annual data for:  
+ Unemployment Rate  
+ GDP  
+ Educational Awards  
+ Occupational Employment  
+ Industry Employment

In [2]:
#packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [3]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

Unemployment Rate: 

*load the following query: JEQAnnual_LAUS*

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "LAUS" dataset, annual
+ Select the following dimensions: Region, Year 
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)     
+ Under "Year" choose your range (most recently I've done 2000 - recent)
+ Select the value "Labor Force", "Employed", and Unemployed". Unemployment rate is available but we're calculating it here in case we ever want to aggregate anything before calculating a percentage. It's been checked that this is correct and aligns with JobsEQ's data.    
+ Swap rows & columns if needed so "Regions" data is at the top, and "Year" is at the side. 

*After loading the query and downloading, resume here.*  

When you've downloaded this csv, open it and delete the top line, rmake a "Year" double header, unmerge the time and make sure it's copy-pasted across the relevant columns and keep the headers for different datapoints. Delete the data notes at the bottom, and select all of the numerical data and make them "General" format. Save as 'JobsEQ_LAUSAnnual_GeosExceptPlaces.csv'

In [4]:
#import LAUS data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_LAUSAnnual_GeosExceptPlaces.csv')
data.head(3)

,Year,USA,USA.1,USA.2,Tennessee (47),Tennessee (47).1,Tennessee (47).2,MPO,MPO.1,MPO.2,GNRC,GNRC.1,GNRC.2,GNRC Region,GNRC Region.1,GNRC Region.2,"Cheatham County, Tennessee (47021)","Cheatham County, Tennessee (47021).1","Cheatham County, Tennessee (47021).2","Davidson County, Tennessee (47037)","Davidson County, Tennessee (47037).1","Davidson County, Tennessee (47037).2","Dickson County, Tennessee (47043)","Dickson County, Tennessee (47043).1","Dickson County, Tennessee (47043).2","Houston County, Tennessee (47083)","Houston County, Tennessee (47083).1","Houston County, Tennessee (47083).2","Humphreys County, Tennessee (47085)","Humphreys County, Tennessee (47085).1","Humphreys County, Tennessee (47085).2","Maury County, Tennessee (47119)","Maury County, Tennessee (47119).1","Maury County, Tennessee (47119).2","Montgomery County, Tennessee (47125)","Montgomery County, Tennessee (47125).1","Montgomery County, Tennessee (47125).2","Robertson County, Tennessee (47147)","Robertson County, Tennessee (47147).1","Robertson County, Tennessee (47147).2","Rutherford County, Tennessee (47149)","Rutherford County, Tennessee (47149).1","Rutherford County, Tennessee (47149).2","Stewart County, Tennessee (47161)","Stewart County, Tennessee (47161).1","Stewart County, Tennessee (47161).2","Sumner County, Tennessee (47165)","Sumner County, Tennessee (47165).1","Sumner County, Tennessee (47165).2","Trousdale County, Tennessee (47169)","Trousdale County, Tennessee (47169).1","Trousdale County, Tennessee (47169).2","Williamson County, Tennessee (47187)","Williamson County, Tennessee (47187).1","Williamson County, Tennessee (47187).2","Wilson County, Tennessee (47189)","Wilson County, Tennessee (47189).1","Wilson County, Tennessee (47189).2"
0,Year,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed,Labor Force,Employed,Unemployed
1,2000,143893565,138065883,5827681,2849026,2738785,110241,668134,647631,20502,753959,730195,23763,790723,765674,25048,19513,18982,531,309162,299570,9592,22119,21339,780,3673,3461,212,8882,8445,437,36764,35479,1284,59350,57148,2202,29566,28535,1031,102558,99470,3088,5499,5236,262,70402,68164,2237,3549,3429,119,69927,68208,1719,49752,48203,1549
2,2001,145059940,138106699,6953241,2857659,2728621,129038,671818,647629,24188,758444,730269,28174,795016,765363,29652,19598,18957,641,306620,295925,10695,22203,21250,953,3619,3367,251,8746,8198,548,36572,35093,1478,59841,57262,2579,29998,28749,1249,105013,101216,3796,5592,5297,294,71680,68523,3157,3595,3400,195,71479,69508,1970,50453,48612,1841


In [5]:
#set index to the extraneous "Time" header and transpose, then rename to "Datapoint" and reset index without dropping
data = data.set_index('Year').transpose()
data = data.rename(columns = {'Year': 'Datapoint'}).reset_index(drop = False)

In [6]:
data.head()

Year,index,Datapoint,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,USA,Labor Force,143893565,145059940,146160666,147005871,148071836,149975300,152127646,153578409,155106956,155126715,155523182,155740916,156232399,156528594,157068244,157960658,159791241,161841127,163121138,164906736,162407439,163401845,165971079
1,USA.1,Employed,138065883,138106699,137615237,138129110,139818438,142222730,145000041,146388401,146047744,140696560,140454859,141714416,143548583,144904563,147293813,149540786,151934228,154721774,156709675,158806258,149294638,154624087,159884645
2,USA.2,Unemployed,5827681,6953241,8545428,8876760,8253397,7752570,7127604,7190008,9059212,14430154,15068323,14026499,12683815,11624031,9774431,8419872,7857013,7119352,6411462,6100477,13112801,8777758,6086434
3,Tennessee (47),Labor Force,2849026,2857659,2898947,2904565,2897792,2898489,3014748,3043258,3060778,3032658,3093118,3117492,3096322,3078892,3043573,3074215,3136398,3197031,3257052,3335269,3303230,3323545,3352029
4,Tennessee (47).1,Employed,2738785,2728621,2750423,2744350,2742910,2738223,2856078,2901946,2862093,2725134,2789055,2833278,2849727,2840127,2842540,2902683,2987678,3077514,3143791,3223663,3056709,3174135,3238558


In [7]:
#make sure there's no head or tail on the strings
data['index'] = data['index'].str.strip()
data['Datapoint'] = data['Datapoint'].str.strip()

In [8]:
#split and expand the geography names on the "." (where there were multiple from duplicates because of how we imported it) and take what was before as the name
boop = data['index'].str.split(pat = ".", expand = True)
data['NAME'] = boop[0]

In [9]:
#drop the original column, set the index to NAME and transpose to rename, use custom module, then transpose back and reset index without dropping
data = data.drop(columns = 'index').set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [10]:
data.head()

Year,NAME,Datapoint,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,United States,Labor Force,143893565,145059940,146160666,147005871,148071836,149975300,152127646,153578409,155106956,155126715,155523182,155740916,156232399,156528594,157068244,157960658,159791241,161841127,163121138,164906736,162407439,163401845,165971079
1,United States,Employed,138065883,138106699,137615237,138129110,139818438,142222730,145000041,146388401,146047744,140696560,140454859,141714416,143548583,144904563,147293813,149540786,151934228,154721774,156709675,158806258,149294638,154624087,159884645
2,United States,Unemployed,5827681,6953241,8545428,8876760,8253397,7752570,7127604,7190008,9059212,14430154,15068323,14026499,12683815,11624031,9774431,8419872,7857013,7119352,6411462,6100477,13112801,8777758,6086434
3,Tennessee,Labor Force,2849026,2857659,2898947,2904565,2897792,2898489,3014748,3043258,3060778,3032658,3093118,3117492,3096322,3078892,3043573,3074215,3136398,3197031,3257052,3335269,3303230,3323545,3352029
4,Tennessee,Employed,2738785,2728621,2750423,2744350,2742910,2738223,2856078,2901946,2862093,2725134,2789055,2833278,2849727,2840127,2842540,2902683,2987678,3077514,3143791,3223663,3056709,3174135,3238558


In [11]:
#go from wide to long format with .melt() naming the columns as single column "Time", keeping "Datapoint" and "NAME" as is, and calling the number "Value"
cols = data.columns
data = data.melt(var_name = 'Year', id_vars = ['NAME', 'Datapoint'], value_vars = cols, value_name = 'Value')
data.head(2)

,NAME,Datapoint,Year,Value
0,United States,Labor Force,2000,143893565
1,United States,Employed,2000,138065883


In [12]:
#set index and transpose on datapoint to rename the original categories from JobsEQ, then transpose and reset index again without dropping
data = data.set_index('Datapoint').transpose()
data = data.rename(columns = {'Labor Force': 'LF:Labor Force', 'Employed': 'LF:Employed', 
                              'Unemployed': 'LF:Unemployed'})
data = data.transpose().reset_index(drop = False)

In [13]:
data.head()

,Datapoint,NAME,Year,Value
0,LF:Labor Force,United States,2000,143893565
1,LF:Employed,United States,2000,138065883
2,LF:Unemployed,United States,2000,5827681
3,LF:Labor Force,Tennessee,2000,2849026
4,LF:Employed,Tennessee,2000,2738785


In [14]:
#long to wide
data = pd.pivot(data, index = ['NAME', 'Year'], columns = 'Datapoint', values = 'Value')

In [15]:
cols = list(data.columns)
data[cols] = data[cols].astype(float)

In [16]:
data['LF:Unemployment %'] = (data['LF:Unemployed']/data['LF:Labor Force']) * 100

In [17]:
data = data.reset_index(drop = False)

In [18]:
data.head()

Datapoint,NAME,Year,LF:Employed,LF:Labor Force,LF:Unemployed,LF:Unemployment %
0,"Cheatham County, Tennessee",2000,18982.0,19513.0,531.0,2.721263
1,"Cheatham County, Tennessee",2001,18957.0,19598.0,641.0,3.270742
2,"Cheatham County, Tennessee",2002,19042.0,19816.0,774.0,3.905935
3,"Cheatham County, Tennessee",2003,18934.0,19756.0,821.0,4.155700
4,"Cheatham County, Tennessee",2004,19153.0,20054.0,900.0,4.487883


In [19]:
#make sure year is formatted as an integer
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [20]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Datapoint                  LF:Employed                                                                                                      \
Year                              2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
NAME                                                                                                                                         
Cheatham County, Tennessee     18982.0   18957.0   19042.0   18934.0   19153.0   19307.0   20020.0   19629.0   19332.0   18409.0   18379.0   
Davidson County, Tennessee    299570.0  295925.0  292120.0  289174.0  288420.0  287713.0  296913.0  310958.0  307246.0  293470.0  314127.0   

Datapoint                                                                                                                                           \
Year                            2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
NAME                                                                                                                                                 
Cheatham County, Tennessee   18533.0   18667.0   18692.0   18764.0   18952.0   19462.0   20229.0   20767.0   21352.0   20166.0   21264.0   22006.0   
Davidson County, Tennessee  322412.0  330508.0  334535.0  339009.0  349099.0  360600.0  374318.0  383885.0  393374.0  366810.0  386521.0  401294.0   

Datapoint                  LF:Labor Force                                                                                                      \
Year                                 2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
NAME                                                                                                                                            
Cheatham County, Tennessee        19513.0   19598.0   19816.0   19756.0   20054.0   20186.0   20880.0   20405.0   20431.0   20262.0   20220.0   
Davidson County, Tennessee       309162.0  306620.0  305054.0  302814.0  301955.0  301278.0  310198.0  323213.0  324564.0  321138.0  342567.0   

Datapoint                                                                                                                                           \
Year                            2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
NAME                                                                                                                                                 
Cheatham County, Tennessee   20273.0   20144.0   20007.0   19857.0   19907.0   20271.0   20868.0   21353.0   21923.0   21386.0   21987.0   22607.0   
Davidson County, Tennessee  349109.0  352776.0  355490.0  356981.0  364943.0  374154.0  385090.0  394301.0  403493.0  399871.0  404212.0  412906.0   

Datapoint                  LF:Unemployed                                                                                                     \
Year                                2000     2001     2002     2003     2004     2005     2006     2007     2008     2009     2010     2011   
NAME                                                                                                                                          
Cheatham County, Tennessee         531.0    641.0    774.0    821.0    900.0    878.0    860.0    775.0   1099.0   1852.0   1840.0   1740.0   
Davidson County, Tennessee        9592.0  10695.0  12934.0  13639.0  13534.0  13564.0  13284.0  12254.0  17317.0  27668.0  28440.0  26697.0   

Datapoint                                                                                                                     LF:Unemployment %  \
Year                           2012     2013     2014     2015     2016     2017     2018     2019     2020     2021     2022              2000   
NAME                                               

In [21]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

LF:Employed                                                                                                      \
                                  2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
                                  None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                         
Cheatham County, Tennessee     18982.0   18957.0   19042.0   18934.0   19153.0   19307.0   20020.0   19629.0   19332.0   18409.0   18379.0   
Davidson County, Tennessee    299570.0  295925.0  292120.0  289174.0  288420.0  287713.0  296913.0  310958.0  307246.0  293470.0  314127.0   
Dickson County, Tennessee      21339.0   21250.0   21309.0   21381.0   21431.0   21503.0   22390.0   22268.0   22011.0   20858.0   21488.0   

                                                                                                                                                    \
                                2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
                                None      None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                                 
Cheatham County, Tennessee   18533.0   18667.0   18692.0   18764.0   18952.0   19462.0   20229.0   20767.0   21352.0   20166.0   21264.0   22006.0   
Davidson County, Tennessee  322412.0  330508.0  334535.0  339009.0  349099.0  360600.0  374318.0  383885.0  393374.0  366810.0  386521.0  401294.0   
Dickson County, Tennessee    21913.0   22121.0   22048.0   22142.0   22841.0   23714.0   24697.0   25550.0   26372.0   24793.0   26090.0   27093.0   

                           LF:Labor Force                                                                                                      \
                                     2000      2001      2002      2003      2004      2005      2006      2007      2008      2009      2010   
                                     None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                            
Cheatham County, Tennessee        19513.0   19598.0   19816.0   19756.0   20054.0   20186.0   20880.0   20405.0   20431.0   20262.0   20220.0   
Davidson County, Tennessee       309162.0  306620.0  305054.0  302814.0  301955.0  301278.0  310198.0  323213.0  324564.0  321138.0  342567.0   
Dickson County, Tennessee         22119.0   22203.0   22375.0   22487.0   22592.0   22620.0   23491.0   23225.0   23622.0   23357.0   23980.0   

                                                                                                                                                    \
                                2011      2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
                                None      None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                                 
Cheatham County, Tennessee   20273.0   20144.0   20007.0   19857.0   19907.0   20271.0   20868.0   21353.0   21923.0   21386.0   21987.0   22607.0   
Davidson County, Tennessee  349109.0  352776.0  355490.0  356981.0  364943.0  374154.0  385090.0  394301.0  403493.0  399871.0  404212.0  412906.0   
Dickson County, Tennessee    24270.0   24166.0   23876.0   23627.

In [22]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
first_level

['LF:Employed', 'LF:Labor Force', 'LF:Unemployed', 'LF:Unemployment %']

In [23]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [24]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [25]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ LAUS'

In [26]:
#it's weird to look at percent change of a percent so dropping the % change as to not confuse anyone later
#also renaming the real change, this is one of the only places it's applicable because normally we wouldn't run change metrics on a percent but want to for ue
data = data.drop(columns = ['LF:Unemployment % % Change'])
data = data.rename(columns = {'LF:Unemployment % Change': 'LF:Unemployment % Real Change'})

In [27]:
#final check
data.head()

,NAME,Year,Time Frame,LF:Employed,LF:Employed % Change,LF:Employed Change,LF:Labor Force,LF:Labor Force % Change,LF:Labor Force Change,LF:Unemployed,LF:Unemployed % Change,LF:Unemployed Change,LF:Unemployment %,LF:Unemployment % Real Change,GEO_ID,Source
0,"Cheatham County, Tennessee",2000,None,18982.0,NaN,NaN,19513.0,NaN,NaN,531.0,NaN,NaN,2.721263,NaN,0500000US47021,JobsEQ LAUS
1,"Cheatham County, Tennessee",2001,None,18957.0,NaN,NaN,19598.0,NaN,NaN,641.0,NaN,NaN,3.270742,NaN,0500000US47021,JobsEQ LAUS
2,"Cheatham County, Tennessee",2002,None,19042.0,NaN,NaN,19816.0,NaN,NaN,774.0,NaN,NaN,3.905935,NaN,0500000US47021,JobsEQ LAUS
3,"Cheatham County, Tennessee",2003,None,18934.0,NaN,NaN,19756.0,NaN,NaN,821.0,NaN,NaN,4.155700,NaN,0500000US47021,JobsEQ LAUS
4,"Cheatham County, Tennessee",2004,None,19153.0,NaN,NaN,20054.0,NaN,NaN,900.0,NaN,NaN,4.487883,NaN,0500000US47021,JobsEQ LAUS


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5244 entries, 0 to 5243
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   NAME                           5244 non-null   object 
 1   Year                           5244 non-null   object 
 2   Time Frame                     5244 non-null   object 
 3   LF:Employed                    437 non-null    float64
 4   LF:Employed % Change           4807 non-null   float64
 5   LF:Employed Change             4807 non-null   float64
 6   LF:Labor Force                 437 non-null    float64
 7   LF:Labor Force % Change        4807 non-null   float64
 8   LF:Labor Force Change          4807 non-null   float64
 9   LF:Unemployed                  437 non-null    float64
 10  LF:Unemployed % Change         4807 non-null   float64
 11  LF:Unemployed Change           4807 non-null   float64
 12  LF:Unemployment %              437 non-null    f

In [29]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_LAUS_Annual_Change', conn, if_exists = 'replace', index = False)

5244

GDP:  

*load the following query: JEQAnnual_GDP*

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "GDP" dataset  
+ Select the following dimensions: Region, Year 
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)     
+ Under "Year" choose your range (most recently I've selected the full available range of 2001 - recent **Watch out here - keep old year if range changes and add onto new download**)  
+ Select the value "GDP".  
+ Keep region on the side and year on the top.

*After loading the query and downloading, resume here.*

When you've downloaded this csv, open it and replace the title row with the year (as all of the values are GDPs), delete the header line and data notes at the bottom, rename region to "NAME", and select all of the numerical data and make them "General" format. You can also quickly delimit the NAMEs to remove the ", Tennessee" off of the end of the counties and change "USA" to "US". This can also be done here, of course. Save as 'JobsEQ_GDPAnnual_GeosExceptPlaces.csv'

In [30]:
#GDP import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_GDPAnnual_GeosExceptPlaces.csv')
data.head(2)

,NAME,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,US,1.058190e+13,1.092910e+13,1.145640e+13,1.221720e+13,1.303920e+13,1.381560e+13,1.447420e+13,1.476980e+13,1.447810e+13,1.504900e+13,1.559970e+13,1.625400e+13,1.684320e+13,1.755070e+13,1.820570e+13,1.869490e+13,1.947960e+13,2.052680e+13,2.137250e+13,2.089370e+13,2.299610e+13
1,Tennessee,1.864350e+11,1.957600e+11,2.051200e+11,2.193220e+11,2.316270e+11,2.435780e+11,2.486910e+11,2.542420e+11,2.523330e+11,2.577760e+11,2.698140e+11,2.863420e+11,2.958630e+11,3.061530e+11,3.252940e+11,3.364140e+11,3.498380e+11,3.613800e+11,3.769150e+11,3.695720e+11,4.182940e+11


In [31]:
data['NAME'].unique()

array(['US', 'Tennessee', 'MPO', 'GNRC', 'GNRC Region', 'Cheatham County',
       'Davidson County', 'Dickson County', 'Houston County',
       'Humphreys County', 'Maury County', 'Montgomery County',
       'Robertson County', 'Rutherford County', 'Stewart County',
       'Sumner County', 'Trousdale County', 'Williamson County',
       'Wilson County'], dtype=object)

In [32]:
#make a list of the columns to melt from wide to long format with the year as a variable and naming the value "GDP"
cols = data.columns
data = data.melt(var_name = 'Year', id_vars = ['NAME'], value_vars = cols, value_name = 'GDP:GDP')

In [33]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [34]:
data.head()

,NAME,Year,GDP:GDP
0,United States,2001,10581900000000.0
1,Tennessee,2001,186435000000.0
2,MPO,2001,53769753000.0
3,GNRC,2001,55545232000.0
4,GNRC Region,2001,58130905000.0


In [35]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [36]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

GDP:GDP                                                                                                    \
Year                                2001          2002          2003          2004          2005          2006          2007          2008   
NAME                                                                                                                                         
Cheatham County, Tennessee  4.590110e+08  5.296220e+08  5.643490e+08  6.247250e+08  6.785940e+08  7.196860e+08  7.098450e+08  7.842820e+08   
Davidson County, Tennessee  3.403081e+10  3.512616e+10  3.727170e+10  3.900497e+10  4.139817e+10  4.386193e+10  4.472193e+10  4.653565e+10   

                                                                                                                                            \
Year                                2009          2010          2011          2012          2013          2014          2015          2016   
NAME                                                                                                                                         
Cheatham County, Tennessee  7.557930e+08  7.694610e+08  7.600230e+08  7.668240e+08  8.288380e+08  8.198790e+08  9.209110e+08  9.394920e+08   
Davidson County, Tennessee  4.719965e+10  4.855240e+10  5.124097e+10  5.435087e+10  5.613963e+10  5.925130e+10  6.270032e+10  6.553752e+10   

                                                                                                  
Year                                2017          2018          2019          2020          2021  
NAME                                                                                              
Cheatham County, Tennessee  1.026863e+09  1.071879e+09  1.104530e+09  1.094723e+09  1.233382e+09  
Davidson County, Tennessee  6.892946e+10  7.293132e+10  7.662392e+10  7.174367e+10  8.255004e+10

In [37]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

GDP:GDP                                                                                                    \
                                    2001          2002          2003          2004          2005          2006          2007          2008   
                                    None          None          None          None          None          None          None          None   
NAME                                                                                                                                         
Cheatham County, Tennessee  4.590110e+08  5.296220e+08  5.643490e+08  6.247250e+08  6.785940e+08  7.196860e+08  7.098450e+08  7.842820e+08   
Davidson County, Tennessee  3.403081e+10  3.512616e+10  3.727170e+10  3.900497e+10  4.139817e+10  4.386193e+10  4.472193e+10  4.653565e+10   
Dickson County, Tennessee   8.536360e+08  8.364840e+08  9.001090e+08  9.450570e+08  1.030411e+09  1.080384e+09  1.092047e+09  1.042446e+09   

                                                                                                                                            \
                                    2009          2010          2011          2012          2013          2014          2015          2016   
                                    None          None          None          None          None          None          None          None   
NAME                                                                                                                                         
Cheatham County, Tennessee  7.557930e+08  7.694610e+08  7.600230e+08  7.668240e+08  8.288380e+08  8.198790e+08  9.209110e+08  9.394920e+08   
Davidson County, Tennessee  4.719965e+10  4.855240e+10  5.124097e+10  5.435087e+10  5.613963e+10  5.925130e+10  6.270032e+10  6.553752e+10   
Dickson County, Tennessee   1.002546e+09  1.019388e+09  1.098554e+09  1.159218e+09  1.291117e+09  1.350150e+09  1.449488e+09  1.532418e+09   

                                                                                                  
                                    2017          2018          2019          2020          2021  
                                    None          None          None          None          None  
NAME                                                                                              
Cheatham County, Tennessee  1.026863e+09  1.071879e+09  1.104530e+09  1.094723e+09  1.233382e+09  
Davidson County, Tennessee  6.892946e+10  7.293132e+10  7.662392e+10  7.174367e+10  8.255004e+10  
Dickson County, Tennessee   1.616509e+09  1.638125e+09  1.750963e+09  1.713265e+09  1.880187e+09

In [38]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
first_level

['GDP:GDP']

In [39]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [40]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [41]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ BEA'

In [42]:
#final check
data.head()

,NAME,Year,Time Frame,GDP:GDP,GDP:GDP % Change,GDP:GDP Change,GEO_ID,Source
0,"Cheatham County, Tennessee",2001,None,459011000.0,NaN,NaN,0500000US47021,JobsEQ BEA
1,"Cheatham County, Tennessee",2002,None,529622000.0,NaN,NaN,0500000US47021,JobsEQ BEA
2,"Cheatham County, Tennessee",2003,None,564349000.0,NaN,NaN,0500000US47021,JobsEQ BEA
3,"Cheatham County, Tennessee",2004,None,624725000.0,NaN,NaN,0500000US47021,JobsEQ BEA
4,"Cheatham County, Tennessee",2005,None,678594000.0,NaN,NaN,0500000US47021,JobsEQ BEA


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4389 entries, 0 to 4388
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NAME              4389 non-null   object 
 1   Year              4389 non-null   object 
 2   Time Frame        4389 non-null   object 
 3   GDP:GDP           399 non-null    float64
 4   GDP:GDP % Change  3990 non-null   float64
 5   GDP:GDP Change    3990 non-null   float64
 6   GEO_ID            3465 non-null   object 
 7   Source            4389 non-null   object 
dtypes: float64(3), object(5)
memory usage: 274.4+ KB


In [44]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_GDP_Annual_Change', conn, if_exists = 'replace', index = False)

4389

Awards:  

*load the following query: JEQAnnual_Awards*

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "Education" dataset  
+ Select the following dimensions: Region and Schools, Year, and Award Level   
+ Under "Region" select "GNRC Region - All Schools", and then sub counties    
+ Under "Year" choose your range (most recently I've selected the full available range of 2010 - recent **Watch out here - keep old year if range changes and add onto new download**)  
+ Under Award Level leave "All Award Levels", and then all subs  
+ Select the value "Awards"  
+ Swap rows & columns so the "Regions & Schools" data is at the top, "Award Level" is on the side, as well as "Year"  

*After loading the query and downloading, resume here*  

When you've downloaded this csv, open it and replace the title row with the geography (as all of the values are all awards) while keeping the headers for award level and year, then delete the top line and data notes at the bottom, and select all of the numerical data and make them "General" format. Save as 'JobsEQ_AwardsAnnual_GeosExceptPlaces.csv'

In [45]:
#Awards import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_AwardsAnnual_GeosExceptPlaces.csv')
data.head(2)

,Award Level,Year,GNRC Region-All Schools,"Cheatham County, Tennessee-All Schools (47021)","Davidson County, Tennessee-All Schools (47037)","Dickson County, Tennessee-All Schools (47043)","Houston County, Tennessee-All Schools (47083)","Humphreys County, Tennessee-All Schools (47085)","Maury County, Tennessee-All Schools (47119)","Montgomery County, Tennessee-All Schools (47125)","Robertson County, Tennessee-All Schools (47147)","Rutherford County, Tennessee-All Schools (47149)","Stewart County, Tennessee-All Schools (47161)","Sumner County, Tennessee-All Schools (47165)","Trousdale County, Tennessee-All Schools (47169)","Williamson County, Tennessee-All Schools (47187)","Wilson County, Tennessee-All Schools (47189)"
0,All Award Levels,2010,25779,NaN,15864,294.0,NaN,NaN,699.0,2166.0,NaN,4891,NaN,1033.0,242.0,55.0,535.0
1,All Award Levels,2011,28038,NaN,17182,394.0,NaN,NaN,676.0,2517.0,NaN,5338,NaN,1033.0,249.0,96.0,553.0


In [47]:
#make a list of columns to take from wide to long format naming the column of current columns "NAME" and the value "Awards"
cols = data.columns
data = data.melt(var_name = 'NAME', id_vars = ['Award Level', 'Year'], value_vars = cols, value_name = 'Awards')
data.head(2)

,Award Level,Year,NAME,Awards
0,All Award Levels,2010,GNRC Region-All Schools,25779.0
1,All Award Levels,2011,GNRC Region-All Schools,28038.0


In [48]:
#split the NAME on common delimiters and expand the split into a new dataframe, where you'll take the first instance and write-over the original "NAME" and strip
boop = data['NAME'].str.split(r"\-|,", expand = True)
data['NAME'] = boop[0].str.strip()

In [49]:
#make lists of years and geographies to loop through and group the different award levels
years = list(data['Year'].unique())
geos = list(data['NAME'].unique())

In [50]:
#format the column headers so that each individual instance can be looped through, and you can convert to float types before by dropping null then converting
data = data.set_index(['Award Level', 'Year', 'NAME']).transpose()
data.head()

Award Level All Award Levels                                                                                                              \
Year                    2010        2011        2012        2013        2014        2015        2016        2017        2018        2019   
NAME             GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards               25779.0     28038.0     30017.0     28404.0     28353.0     28549.0     28580.0     28689.0     27726.0     27785.0   

Award Level                         Certificate < 1 Yr                                                                                      \
Year               2020        2021               2010        2011        2012        2013        2014        2015        2016        2017   
NAME        GNRC Region GNRC Region        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards          28519.0     29332.0             2404.0      2065.0      2891.0      2070.0      2192.0      2370.0      2357.0      2479.0   

Award Level                                                 Certificate 1+ but < 2 Yr                                                              \
Year               2018        2019        2020        2021                      2010        2011        2012        2013        2014        2015   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region               GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards           1644.0      1949.0      2048.0      2251.0                    2551.0      4466.0      5179.0      4114.0      3697.0      3549.0   

Award Level                                                                         Associate's                                                  \
Year               2016        2017        2018        2019        2020        2021        2010        2011        2012        2013        2014   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards           3418.0      3304.0      3107.0      2738.0      1824.0      2063.0      3416.0      4153.0      4459.0      4008.0      3644.0   

Award Level                                                                                     Certificate 2+ but < 4 Yr                          \
Year               2015        2016        2017        2018        2019        2020        2021                      2010        2011        2012   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region               GNRC Region GNRC Region GNRC Region   
Awards           3818.0      3749.0      3766.0      4010.0      4160.0      4849.0      5212.0                    1293.0       644.0       177.0   

Award Level                                                                                                              Bachelor's              \
Year               2013        2014        2015        2016        2017        2018        2019        2020        2021        2010        2011   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards            118.0       254.0       198.0       253.0       186.0       193.0       234.0       640.0       634.0     10772.0     11215.0   

Award Level                                                                                                                          \
Year               2012        2013        2014        2015        2016        2017        2018        2019        2020        2021   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards          11364.0     11904.0     12237.0     12252.0     12501.0     12637.0     12565

In [51]:
#fill nulls with 0.0 and convert to float
data = data.fillna(0.0)
cols = data.columns
data[cols] = data[cols].astype(float)

In [52]:
#loop through all columns grouping certain certificates and higher levels of awards for each geography and year then transpose & reset idnex without dropping
for year in years:
    for geo in geos:
        data[('Certificate < 2 Years', year, '{}'.format(geo))] = data[('Certificate < 1 Yr', year, '{}'.format(geo))] + data[('Certificate 1+ but < 2 Yr',
                                                                                                                                  year, '{}'.format(geo))]
        data[('Certificate > 2 Years', year, '{}'.format(geo))] = data[('Certificate 2+ but < 4 Yr', year, '{}'.format(geo))]
        thelist = [data[("Master's", year, '{}'.format(geo))], data[("Post-Master's", year, '{}'.format(geo))], 
                   data[("Doctorate", year, '{}'.format(geo))]]
        data[("Master's or Above", year, '{}'.format(geo))] = sum(thelist)
        thelist = [data[("Bachelor's", year, '{}'.format(geo))], data[("Post-Baccalaureate", year, '{}'.format(geo))]]
        data[("Bachelor's", year, '{}'.format(geo))] = sum(thelist)
        data = data.drop(columns = [('Certificate < 1 Yr', year, '{}'.format(geo)), ('Certificate 1+ but < 2 Yr', year, '{}'.format(geo)), 
                                      ('Certificate 2+ but < 4 Yr', year, '{}'.format(geo)), ('Post-Baccalaureate', year, '{}'.format(geo))])

In [53]:
#transpose the dataframe and reset the index without dropping
data = data.transpose().reset_index(drop = False)

In [54]:
data.head()

,Award Level,Year,NAME,Awards
0,All Award Levels,2010,GNRC Region,25779.0
1,All Award Levels,2011,GNRC Region,28038.0
2,All Award Levels,2012,GNRC Region,30017.0
3,All Award Levels,2013,GNRC Region,28404.0
4,All Award Levels,2014,GNRC Region,28353.0


In [55]:
#pivot to take from long to wide format on award levels
data = data.pivot(index = ['Year', 'NAME'], columns = 'Award Level')['Awards']

In [56]:
data.head()

Award Level           All Award Levels  Associate's  Bachelor's  Certificate < 2 Years  Certificate > 2 Years  Doctorate  Master's  \
Year NAME                                                                                                                            
2010 Cheatham County               0.0          0.0         0.0                    0.0                    0.0        0.0       0.0   
     Davidson County           15864.0       1711.0      5742.0                 3221.0                 1164.0      958.0    3045.0   
     Dickson County              294.0          0.0         0.0                  248.0                   46.0        0.0       0.0   
     GNRC Region               25779.0       3416.0     10823.0                 4955.0                 1293.0      978.0    4151.0   
     Houston County                0.0          0.0         0.0                    0.0                    0.0        0.0       0.0   

Award Level           Master's or Above  Post-Master's  
Year NAME                                               
2010 Cheatham County                0.0            0.0  
     Davidson County             4026.0           23.0  
     Dickson County                 0.0            0.0  
     GNRC Region                 5292.0          163.0  
     Houston County                 0.0            0.0

In [57]:
#reset and remove name from index
data = data.reset_index(drop = False).rename_axis(None, axis=1).reset_index(drop = True)

In [58]:
#set the index as year and name to remove them from the set of columns for prefix, add the prefix, then drop and keep the indices
data.set_index(['Year', 'NAME'], inplace = True)
data = data.add_prefix('Awards:').reset_index(drop = False)

In [59]:
#calculate percentages
data["Awards %:Associate's"] = percent(data["Awards:Associate's"], data['Awards:All Award Levels'])
data["Awards %:Bachelor's"] = percent(data["Awards:Bachelor's"], data['Awards:All Award Levels'])
data["Awards %:Certificate < 2 Years"] = percent(data["Awards:Certificate < 2 Years"], data['Awards:All Award Levels'])
data["Awards %:Certificate > 2 Years"] = percent(data["Awards:Certificate > 2 Years"], data['Awards:All Award Levels'])
data["Awards %:Doctorate"] = percent(data["Awards:Doctorate"], data['Awards:All Award Levels'])
data["Awards %:Master's"] = percent(data["Awards:Master's"], data['Awards:All Award Levels'])
data["Awards %:Master's or Above"] = percent(data["Awards:Master's or Above"], data['Awards:All Award Levels'])
data["Awards %:Post-Master's"] = percent(data["Awards:Post-Master's"], data['Awards:All Award Levels'])

In [60]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [61]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [62]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Awards %:Associate's                                                                                              \
Year                                       2010       2011      2012       2013      2014      2015      2016      2017      2018      2019   
NAME                                                                                                                                          
Cheatham County, Tennessee                  NaN        NaN       NaN        NaN  0.000000  0.000000  0.000000       NaN       NaN       NaN   
Davidson County, Tennessee            10.785426  12.903038  13.32947  11.077863  9.585555  9.653643  9.143992  8.610736  9.004679  9.094468   

                                               Awards %:Bachelor's                                                                             \
Year                            2020      2021                2010      2011       2012       2013       2014       2015      2016       2017   
NAME                                                                                                                                            
Cheatham County, Tennessee       NaN       NaN                 NaN       NaN        NaN        NaN   0.000000   0.000000   0.00000        NaN   
Davidson County, Tennessee  8.499967  8.931766           36.195159  34.27424  33.387424  36.635115  38.075618  38.704458  39.86877  40.093355   

                                                                       Awards %:Certificate < 2 Years                                               \
Year                             2018       2019       2020       2021                           2010       2011       2012       2013        2014   
NAME                                                                                                                                                 
Cheatham County, Tennessee        NaN        NaN        NaN        NaN                            NaN        NaN        NaN        NaN  100.000000   
Davidson County, Tennessee  42.267513  42.660491  43.588728  42.793253                      20.303833  27.034105  26.850188  23.847328   22.424867   

                                                                                                          Awards %:Certificate > 2 Years            \
Year                              2015        2016       2017       2018       2019       2020       2021                           2010      2011   
NAME                                                                                                                                                 
Cheatham County, Tennessee  100.000000  100.000000        NaN        NaN        NaN        NaN        NaN                            NaN       NaN   
Davidson County, Tennessee   22.183605   21.093186  21.594399  18.086265  16.492693  11.746849  12.854587                       7.337368  2.322198   

                                                                                                                             Awards %:Doctorate  \
Year                            2012      2013     2014      2015     2016      2017      2018      2019      2020      2021               2010   
NAME                                                                                                                                              
Cheatham County, Tennessee       NaN       NaN  0.00000  0.000000  0.00000       NaN       NaN       NaN       NaN       NaN                NaN   
Davidson County, Tennessee  0.365112  0.244275  0.27872  0.269655  0.70431  0.319371  0.378132  0.685021  3.339273  3.002811            6.03883   

                                                                                                                                            \
Year                            2011      2012      2013      2014      2015      2016      2017      2018      2019       2020       2021   
NAME                                                           

In [63]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(2)

Awards %:Associate's                                                                                              \
                                           2010       2011      2012       2013      2014      2015      2016      2017      2018      2019   
                                           None       None      None       None      None      None      None      None      None      None   
NAME                                                                                                                                          
Cheatham County, Tennessee                  NaN        NaN       NaN        NaN  0.000000  0.000000  0.000000       NaN       NaN       NaN   
Davidson County, Tennessee            10.785426  12.903038  13.32947  11.077863  9.585555  9.653643  9.143992  8.610736  9.004679  9.094468   

                                               Awards %:Bachelor's                                                                             \
                                2020      2021                2010      2011       2012       2013       2014       2015      2016       2017   
                                None      None                None      None       None       None       None       None      None       None   
NAME                                                                                                                                            
Cheatham County, Tennessee       NaN       NaN                 NaN       NaN        NaN        NaN   0.000000   0.000000   0.00000        NaN   
Davidson County, Tennessee  8.499967  8.931766           36.195159  34.27424  33.387424  36.635115  38.075618  38.704458  39.86877  40.093355   

                                                                       Awards %:Certificate < 2 Years                                               \
                                 2018       2019       2020       2021                           2010       2011       2012       2013        2014   
                                 None       None       None       None                           None       None       None       None        None   
NAME                                                                                                                                                 
Cheatham County, Tennessee        NaN        NaN        NaN        NaN                            NaN        NaN        NaN        NaN  100.000000   
Davidson County, Tennessee  42.267513  42.660491  43.588728  42.793253                      20.303833  27.034105  26.850188  23.847328   22.424867   

                                                                                                          Awards %:Certificate > 2 Years            \
                                  2015        2016       2017       2018       2019       2020       2021                           2010      2011   
                                  None        None       None       None       None       None       None                           None      None   
NAME                                                                                                                                                 
Cheatham County, Tennessee  100.000000  100.000000        NaN        NaN        NaN        NaN        NaN                            NaN       NaN   
Davidson County, Tennessee   22.183605   21.093186  21.594399  18.086265  16.492693  11.746849  12.854587                       7.337368  2.322198   

                                                                                                                             Awards %:Doctorate  \
                                2012      2013     2014      2015     2016      2017      2018      2019      2020      2021               2010   
                                None      None     None      None     None      None      None      None      None      None               None   
NAME                                                  

In [64]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [65]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [66]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [67]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ NCES'

In [68]:
#final check
data.head(3)

,NAME,Year,Time Frame,Awards %:Associate's,Awards %:Bachelor's,Awards %:Certificate < 2 Years,Awards %:Certificate > 2 Years,Awards %:Doctorate,Awards %:Master's,Awards %:Master's or Above,Awards %:Post-Master's,Awards:All Award Levels,Awards:All Award Levels % Change,Awards:All Award Levels Change,Awards:Associate's,Awards:Associate's % Change,Awards:Associate's Change,Awards:Bachelor's,Awards:Bachelor's % Change,Awards:Bachelor's Change,Awards:Certificate < 2 Years,Awards:Certificate < 2 Years % Change,Awards:Certificate < 2 Years Change,Awards:Certificate > 2 Years,Awards:Certificate > 2 Years % Change,Awards:Certificate > 2 Years Change,Awards:Doctorate,Awards:Doctorate % Change,Awards:Doctorate Change,Awards:Master's,Awards:Master's % Change,Awards:Master's Change,Awards:Master's or Above,Awards:Master's or Above % Change,Awards:Master's or Above Change,Awards:Post-Master's,Awards:Post-Master's % Change,Awards:Post-Master's Change,GEO_ID,Source
0,"Cheatham County, Tennessee",2010,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0500000US47021,JobsEQ NCES
1,"Cheatham County, Tennessee",2011,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0500000US47021,JobsEQ NCES
2,"Cheatham County, Tennessee",2012,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0500000US47021,JobsEQ NCES


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 40 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   NAME                                   1170 non-null   object 
 1   Year                                   1170 non-null   object 
 2   Time Frame                             1170 non-null   object 
 3   Awards %:Associate's                   123 non-null    float64
 4   Awards %:Bachelor's                    123 non-null    float64
 5   Awards %:Certificate < 2 Years         123 non-null    float64
 6   Awards %:Certificate > 2 Years         123 non-null    float64
 7   Awards %:Doctorate                     123 non-null    float64
 8   Awards %:Master's                      123 non-null    float64
 9   Awards %:Master's or Above             123 non-null    float64
 10  Awards %:Post-Master's                 123 non-null    float64
 11  Awar

In [70]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_Awards_Annual_Change', conn, if_exists = 'replace', index = False)

1170

This retrieves historical and current annual occupational employment disaggregated by high level occupation group.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  

*load the following query: JEQAnnual_Occupation*

+ Select the "Occupation" dataset, year/quarter is the only option, we'll have to make it into an annual number  
+ Select parameters: "Region", "Occupation", "Year/Quarter"  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Places"  
+ For "Year/Quarter" select as many as possible  
+ Under "Occupation" expand "Total-All Occupations" and select the combination of 2 digit SOCS codes and built groups in Jobs EQ as described below, leaving Total as well  

Occupations to Select:  
(SOC Codes)  
+ Total - All Occupations  
(Custom Created Groups) 
+ SnapshotOcc 11: Healthcare Practitioners and Technical (29: Healthcare Practitioners and Technical Occupations)  
+ SnapshotOcc 11: Healthcare Support (31: Healthcare Support Occupations)  
+ SnapshotOcc 11: Protective Service (33: Protective Service Occupations)  
+ SnapshotOcc 11: Production (51: Production Occupations)  
+ SnapshotOcc 11: Transportation and Material Moving (53: Transportation and Material Moving Occupations)   
+ SnapshotOcc 11: Business & Financial (11: Management & 13: Business & Financial Operations)  
+ SnapshotOcc 11: Computer, Mathematical, & Engineering (15: Computer & Mathematical, 17: Architecture & Engineering, & 19: Life, Physical, & Social Science)  
+ SnapshotOcc 11: Education, Legal, Social, Arts, & Media (21: Community & Social Service, 23: Legal, 25: Educational Instruction & Library, & 27: Art, Design, Entertainment, Sports, & Media)  
+ SnapshotOcc 11: Food, Property, & Personal Services (35: Food Preparation & Serving, 37: Building & Grounds Cleaning & Maintenance, & 39: Personal Care & Service)  
+ SnapshotOcc 11: Sales & Office Support (41: Sales & Related & 43: Office & Administrative Support)  
+ SnapshotOcc 11: Natural Resources, Construction, & Maintenance (45: Farming Fishing & Forestry, 47: Construction & Extraction, & 49: Installation Maintenance & Repair)

+ Make sure Occupation Data is on the top, and Year/Quarter and Region are on the side (this will make the small bit of Excel formatting easier)  
+ Select value: Employment, place of work  

This may exceed the drafting size, in which case you'll have to queue for download. 

*After loading the query and downloading, resume here*  

Once it is completed, open and delete the top row that says "Occupation Data", and delete the third row that repeats "Employment (Place of Work)". Rename the time indicator column to Time, and "Region" to "NAME". Select all numerical data and format to "general". Delete metadata at the bottom. Save in the "Data Downloads" folder as "JobsEQ_OccupationalEmploymentAnnual_AllGeos.csv"

In [136]:
#import and examine, dtype str so the GEO_ID formatting doesn't get messed up and so that we can split the time column
data = pd.read_csv('../../Data Downloads/JobsEQ_OccupationalEmploymentAnnual_AllGeos.csv', dtype = str)
data.head(2)

,Time,NAME,Total - All Occupations,SnapshotOcc 11: Production,SnapshotOcc 11: Healthcare Support,SnapshotOcc 11: Protective Services,SnapshotOcc 11: Business & Financial,SnapshotOcc 11: Sales & Office Support,SnapshotOcc 11: Transportation and Material Moving,"SnapshotOcc 11: Food, Property, & Personal Services","SnapshotOcc 11: Computer, Mathematical, & Engineering",SnapshotOcc 11: Healthcare Practitioners and Technical,"SnapshotOcc 11: Education, Legal, Social, Arts, & Media","SnapshotOcc 11: Natural Resources, Construction, & Maintenance"
0,2013Q1,USA,143440904.4,9270804.205,5601117.581,3198792.766,14917065.24,36218398.53,11270093.43,20854724.15,6594921.793,7502249.187,14966834.32,13045903.19
1,2013Q1,Tennessee,2929041.521,250615.801,108861.1784,63659.9016,276936.4256,724714.469,293938.783,411230.5865,100613.9871,172962.1501,272079.4975,253428.7416


In [137]:
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)
data.head(2)

,NAME,Time,Total - All Occupations,SnapshotOcc 11: Production,SnapshotOcc 11: Healthcare Support,SnapshotOcc 11: Protective Services,SnapshotOcc 11: Business & Financial,SnapshotOcc 11: Sales & Office Support,SnapshotOcc 11: Transportation and Material Moving,"SnapshotOcc 11: Food, Property, & Personal Services","SnapshotOcc 11: Computer, Mathematical, & Engineering",SnapshotOcc 11: Healthcare Practitioners and Technical,"SnapshotOcc 11: Education, Legal, Social, Arts, & Media","SnapshotOcc 11: Natural Resources, Construction, & Maintenance"
0,United States,2013Q1,143440904.4,9270804.205,5601117.581,3198792.766,14917065.24,36218398.53,11270093.43,20854724.15,6594921.793,7502249.187,14966834.32,13045903.19
1,Tennessee,2013Q1,2929041.521,250615.801,108861.1784,63659.9016,276936.4256,724714.469,293938.783,411230.5865,100613.9871,172962.1501,272079.4975,253428.7416


In [138]:
#use str.split to break out the Time column and create a year and quarter column
boop = data['Time'].str.split(pat = 'Q', expand = True)
data['Year'] = boop[0]
data['Quarter'] = boop[1]
data.drop(columns = 'Time', inplace = True)

In [139]:
#we want the last full annual number, which is 2021 at this time, so knock out the handful of quarters in 2012 and 2022
data = data.loc[(data['Year'] != '2013')&(data['Year'] != '2023')]

In [140]:
#to average out years, make a list of years and of geographies in the dataframe
years = list(data.Year.unique())
geos = list(data.NAME.unique())

In [141]:
#set the index to be the combination of quarter year and name, as this gives unique instances to loop through, and transpose
data = data.set_index(['Quarter', 'Year', 'NAME']).transpose()

In [142]:
#make sure the other columns are float datatypes to perform aggregations
cols = list(data.columns)
data[cols] = data[cols].astype(float)

In [143]:
#loop through all four quarters, the list of unique years, and unique geographies to take the average of all four quarters
cols = data.columns
for year in years:
    for geo in geos:
        thelist = [data[('1', year, '{}'.format(geo))], data[('2', year, '{}'.format(geo))], data[('3', year, '{}'.format(geo))], 
                  data[('4', year, '{}'.format(geo))]]
        data[('All', year, '{}'.format(geo))] = (sum(thelist))/4
data = data.drop(columns = cols)

In [144]:
#transpose the data and reset the index without dropping, drop the quarter column
data = data.transpose().reset_index(drop = False).drop(columns = 'Quarter')

In [145]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['SnapshotOcc 11: Healthcare Practitioners and Technical'], data['SnapshotOcc 11: Healthcare Support'], 
           data['SnapshotOcc 11: Protective Services'], data['SnapshotOcc 11: Production'], 
           data['SnapshotOcc 11: Transportation and Material Moving'], data['SnapshotOcc 11: Business & Financial'],
           data['SnapshotOcc 11: Computer, Mathematical, & Engineering'], data['SnapshotOcc 11: Education, Legal, Social, Arts, & Media'], 
           data['SnapshotOcc 11: Food, Property, & Personal Services'], data['SnapshotOcc 11: Sales & Office Support'], 
           data['SnapshotOcc 11: Natural Resources, Construction, & Maintenance']]
data['Total - All Occupations'] = sum(thelist)

In [146]:
#rename columns to account for the disparity between custom groups and SOC 2 digit formatting
data = data.rename(columns = {'SnapshotOcc 11: Healthcare Practitioners and Technical': 'Healthcare Practitioners & Technical',
                              'SnapshotOcc 11: Healthcare Support': 'Healthcare Support', 
                              'SnapshotOcc 11: Protective Services': 'Protective Services',
                              'SnapshotOcc 11: Production': 'Production', 
                              'SnapshotOcc 11: Transportation and Material Moving': 'Transportation & Material Moving', 
                              'SnapshotOcc 11: Business & Financial': 'Business & Financial', 
                              'SnapshotOcc 11: Sales & Office Support': 'Sales & Office Support', 
                              'SnapshotOcc 11: Food, Property, & Personal Services': 'Food, Property, & Personal Services', 
                              'SnapshotOcc 11: Computer, Mathematical, & Engineering': 'Computer, Mathematical, & Engineering', 
                              'SnapshotOcc 11: Education, Legal, Social, Arts, & Media': 'Education, Legal, Social, Arts, & Media', 
                              'SnapshotOcc 11: Natural Resources, Construction, & Maintenance': 'Natural Resources, Construction, & Maintenance', 
                             'Total - All Occupations': 'Total'})

In [147]:
years = list(data['Year'].unique())

In [148]:
#set the index to year and name so that we can add a prefix to all of the other columns, before resetting the index without dropping
data = data.set_index(['Year', 'NAME'])
data = data.add_prefix('Occ:')#.reset_index(drop = False)
data = data.transpose()

In [149]:
cols = data.columns
data[cols] = data[cols].astype(float)

In [150]:
for time in years:
        RuthInc = [data[('{}'.format(time), 'Eagleville city, Tennessee')],data[('{}'.format(time), 'La Vergne city, Tennessee')],
               data[('{}'.format(time), 'Murfreesboro city, Tennessee')],data[('{}'.format(time), 'Smyrna town, Tennessee')]]
        data[('{}'.format(time), 'Rutherford Incorporated')] = sum(RuthInc)
        data[('{}'.format(time), 'Rutherford Unincorporated')] = data[('{}'.format(time), 'Rutherford County, Tennessee')] - data[('{}'.format(time), 'Rutherford Incorporated')]
        WilsonInc = [data[('{}'.format(time), 'Lebanon city, Tennessee')],data[('{}'.format(time), 'Mount Juliet city, Tennessee')],
                     data[('{}'.format(time), 'Watertown city, Tennessee')]]
        data[('{}'.format(time), 'Wilson Incorporated')] = sum(WilsonInc)
        data[('{}'.format(time), 'Wilson Unincorporated')] = data[('{}'.format(time), 'Wilson County, Tennessee')] - data[('{}'.format(time), 'Wilson Incorporated')]
        CheathInc = [data[('{}'.format(time), 'Ashland City town, Tennessee')],data[('{}'.format(time), 'Kingston Springs town, Tennessee')],
                     data[('{}'.format(time), 'Pegram town, Tennessee')],data[('{}'.format(time), 'Pleasant View city, Tennessee')]]
        data[('{}'.format(time), 'Cheatham Incorporated')] = sum(CheathInc)
        data[('{}'.format(time), 'Cheatham Unincorporated')] = data[('{}'.format(time), 'Cheatham County, Tennessee')] - data[('{}'.format(time), 'Cheatham Incorporated')]
        DicksInc = [data[('{}'.format(time), 'Burns town, Tennessee')],data[('{}'.format(time), 'Charlotte town, Tennessee')],
                    data[('{}'.format(time), 'Dickson city, Tennessee')],data[('{}'.format(time), 'Slayden town, Tennessee')],
                    data[('{}'.format(time), 'Vanleer town, Tennessee')],data[('{}'.format(time), 'White Bluff town, Tennessee')]]
        data[('{}'.format(time), 'Dickson Incorporated')] = sum(DicksInc)
        data[('{}'.format(time), 'Dickson Unincorporated')] = data[('{}'.format(time), 'Dickson County, Tennessee')] - data[('{}'.format(time), 'Dickson Incorporated')]
        HumphInc = [data[('{}'.format(time), 'McEwen city, Tennessee')],data[('{}'.format(time), 'New Johnsonville city, Tennessee')],
                    data[('{}'.format(time), 'Waverly city, Tennessee')]]
        data[('{}'.format(time), 'Humphreys Incorporated')] = sum(HumphInc)
        data[('{}'.format(time), 'Humphreys Unincorporated')] = data[('{}'.format(time), 'Humphreys County, Tennessee')] - data[('{}'.format(time), 'Humphreys Incorporated')]
        data[('{}'.format(time), 'Montgomery Incorporated')] = data[('{}'.format(time), 'Clarksville city, Tennessee')]
        data[('{}'.format(time), 'Montgomery Unincorporated')] = data[('{}'.format(time), 'Montgomery County, Tennessee')] - data[('{}'.format(time), 'Montgomery Incorporated')]

In [151]:
data = data.transpose()
data.head()

Occ:Total  Occ:Production  Occ:Healthcare Support  Occ:Protective Services  Occ:Business & Financial  \
Year NAME                                                                                                                     
2014 United States  1.466024e+08    9.336933e+06            6.077918e+06             3.240551e+06              1.572345e+07   
     Tennessee      2.987402e+06    2.556443e+05            1.111377e+05             6.444783e+04              2.916820e+05   
     GNRC           9.266254e+05    6.560458e+04            3.356368e+04             2.090939e+04              9.607484e+04   
     MPO            8.673386e+05    6.027256e+04            3.198943e+04             1.991289e+04              9.098220e+04   
     GNRC Region    9.598511e+05    6.893528e+04            3.511183e+04             2.147517e+04              9.940808e+04   

                    Occ:Sales & Office Support  Occ:Transportation & Material Moving  Occ:Food, Property, & Personal Services  \
Year NAME                                                                                                                       
2014 United States                3.637388e+07                          1.164071e+07                             2.111585e+07   
     Tennessee                    7.302210e+05                          3.026366e+05                             4.225645e+05   
     GNRC                         2.400535e+05                          8.146127e+04                             1.322912e+05   
     MPO                          2.270869e+05                          7.630056e+04                             1.217026e+05   
     GNRC Region                  2.482149e+05                          8.406311e+04                             1.371768e+05   

                    Occ:Computer, Mathematical, & Engineering  Occ:Healthcare Practitioners & Technical  \
Year NAME                                                                                                 
2014 United States                               6.925063e+06                              7.715660e+06   
     Tennessee                                   1.072614e+05                              1.771351e+05   
     GNRC                                        3.473702e+04                              5.517022e+04   
     MPO                                         3.297216e+04                              5.330581e+04   
     GNRC Region                                 3.559718e+04                              5.746245e+04   

                    Occ:Education, Legal, Social, Arts, & Media  Occ:Natural Resources, Construction, & Maintenance  
Year NAME                                                                                                            
2014 United States                                 1.506559e+07                                       1.338679e+07   
     Tennessee                                     2.702279e+05                                       2.544440e+05   
     GNRC                                          9.208031e+04                                       7.467940e+04   
     MPO                                           8.447897e+04                                       6.833459e+04   
     GNRC Region                                   9.502522e+04                                       7.738112e+04

In [152]:
data = data.reset_index(drop = False)

In [153]:
#melt this data down using the columns name and year to distinguish specific instances, and then attributing total employment per occupation
#name it separately because it bugs out if you call this data and merge later
cols = data.columns
thing = data.melt(var_name = 'Occupation', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Employment')
thing.head(2)

,NAME,Year,Occupation,Employment
0,United States,2014,Occ:Total,1.466024e+08
1,Tennessee,2014,Occ:Total,2.987402e+06


In [154]:
#create a separate dataframe where we will calculate percent of the total for each occupation group
percents = data

In [155]:
#calculate percent of total
percents['Occ:Healthcare Practitioners & Technical'] = percent(percents['Occ:Healthcare Practitioners & Technical'], percents['Occ:Total'])
percents['Occ:Healthcare Support'] = percent(percents['Occ:Healthcare Support'], percents['Occ:Total'])
percents['Occ:Protective Services'] = percent(percents['Occ:Protective Services'], percents['Occ:Total'])
percents['Occ:Production'] = percent(percents['Occ:Production'], percents['Occ:Total'])
percents['Occ:Transportation & Material Moving'] = percent(percents['Occ:Transportation & Material Moving'], percents['Occ:Total'])
percents['Occ:Business & Financial'] = percent(percents['Occ:Business & Financial'], percents['Occ:Total'])
percents['Occ:Computer, Mathematical, & Engineering'] = percent(percents['Occ:Computer, Mathematical, & Engineering'], percents['Occ:Total'])
percents['Occ:Education, Legal, Social, Arts, & Media'] = percent(percents['Occ:Education, Legal, Social, Arts, & Media'], percents['Occ:Total'])
percents['Occ:Food, Property, & Personal Services'] = percent(percents['Occ:Food, Property, & Personal Services'], percents['Occ:Total'])
percents['Occ:Sales & Office Support'] = percent(percents['Occ:Sales & Office Support'], percents['Occ:Total'])
percents['Occ:Natural Resources, Construction, & Maintenance'] = percent(percents['Occ:Natural Resources, Construction, & Maintenance'], percents['Occ:Total'])

In [156]:
#similarly melt this dataframe
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Occupation', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Percent of Employment')
percents.head()

,NAME,Year,Occupation,Percent of Employment
0,United States,2014,Occ:Total,1.466024e+08
1,Tennessee,2014,Occ:Total,2.987402e+06
2,GNRC,2014,Occ:Total,9.266254e+05
3,MPO,2014,Occ:Total,8.673386e+05
4,GNRC Region,2014,Occ:Total,9.598511e+05


In [157]:
#we never renamed the total column when we duplicated the dataframe for percents, we want this to be null so just transpose, drop it, and transpose back
percents = percents.set_index('Occupation').transpose()
percents = percents.drop(columns = 'Occ:Total')
percents = percents.transpose().reset_index(drop = False)

In [158]:
#merge total employment and percent of total employment
data = thing.merge(percents, on = ['Occupation', 'Year', 'NAME'], how = 'outer')
data.head(3)

,NAME,Year,Occupation,Employment,Percent of Employment
0,United States,2014,Occ:Total,1.466024e+08,NaN
1,Tennessee,2014,Occ:Total,2.987402e+06,NaN
2,GNRC,2014,Occ:Total,9.266254e+05,NaN


In [159]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos before resetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus).reset_index(drop = False)

In [160]:
#set the index to "index" (the variable names) and transpose, then reset the index without dropping it
data = data.set_index('index').transpose().reset_index(drop = False)

In [161]:
data.head()

index,NAME,Year,Occupation,Employment,Percent of Employment
0,United States,2014,Occ:Total,146602396.842,NaN
1,Tennessee,2014,Occ:Total,2987402.25729,NaN
2,GNRC,2014,Occ:Total,926625.38286,NaN
3,MPO,2014,Occ:Total,867338.61267,NaN
4,GNRC Region,2014,Occ:Total,959851.097712,NaN


In [162]:
#go long to wide
data = pd.pivot(data, index = ['NAME', 'Year'], columns = 'Occupation', values = ['Employment', 'Percent of Employment'])

In [163]:
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
data.columns = data.columns.to_flat_index()

In [164]:
#rename
occdict = {('Employment', 'Occ:Business & Financial'): 'Occ:Business & Financial', 
           ('Employment', 'Occ:Computer, Mathematical, & Engineering'): 'Occ:Computer, Mathematical, & Engineering',
           ('Employment', 'Occ:Education, Legal, Social, Arts, & Media'): 'Occ:Education, Legal, Social, Arts, & Media', 
           ('Employment', 'Occ:Food, Property, & Personal Services'): 'Occ:Food, Property, & Personal Services', 
           ('Employment', 'Occ:Healthcare Practitioners & Technical'): 'Occ:Healthcare Practitioners & Technical', 
           ('Employment', 'Occ:Healthcare Support'): 'Occ:Healthcare Support', 
           ('Employment', 'Occ:Natural Resources, Construction, & Maintenance'): 'Occ:Natural Resources, Construction, & Maintenance', 
           ('Employment', 'Occ:Production'): 'Occ:Production', 
           ('Employment', 'Occ:Protective Services'): 'Occ:Protective Services', 
           ('Employment', 'Occ:Sales & Office Support'): 'Occ:Sales & Office Support', 
           ('Employment', 'Occ:Total'): 'Occ:Total', 
           ('Employment', 'Occ:Transportation & Material Moving'): 'Occ:Transportation & Material Moving',
           ('Percent of Employment', 'Occ:Transportation & Material Moving'): 'Occ%:Transportation & Material Moving', 
           ('Percent of Employment', 'Occ:Business & Financial'): 'Occ%:Business & Financial', 
           ('Percent of Employment', 'Occ:Computer, Mathematical, & Engineering'): 'Occ%:Computer, Mathematical, & Engineering',
           ('Percent of Employment', 'Occ:Education, Legal, Social, Arts, & Media'): 'Occ%:Education, Legal, Social, Arts, & Media', 
           ('Percent of Employment', 'Occ:Food, Property, & Personal Services'): 'Occ%:Food, Property, & Personal Services', 
           ('Percent of Employment', 'Occ:Healthcare Practitioners & Technical'): 'Occ%:Healthcare Practitioners & Technical', 
           ('Percent of Employment', 'Occ:Healthcare Support'): 'Occ%:Healthcare Support', 
           ('Percent of Employment', 'Occ:Natural Resources, Construction, & Maintenance'): 'Occ%:Natural Resources, Construction, & Maintenance', 
           ('Percent of Employment', 'Occ:Production'): 'Occ%:Production', 
           ('Percent of Employment', 'Occ:Protective Services'): 'Occ%:Protective Services', 
           ('Percent of Employment', 'Occ:Sales & Office Support'): 'Occ%:Sales & Office Support', 
           ('Percent of Employment', 'Occ:Total'): 'Occ%:Total', 
           ('Percent of Employment', 'Occ:Transportation & Material Moving'): 'Occ%:Transportation & Material Moving' }
data = data.rename(columns = occdict)
data = data.reset_index(drop = False)

In [165]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [166]:
data.head(2)

,NAME,Year,Occ:Business & Financial,"Occ:Computer, Mathematical, & Engineering","Occ:Education, Legal, Social, Arts, & Media","Occ:Food, Property, & Personal Services",Occ:Healthcare Practitioners & Technical,Occ:Healthcare Support,"Occ:Natural Resources, Construction, & Maintenance",Occ:Production,Occ:Protective Services,Occ:Sales & Office Support,Occ:Total,Occ:Transportation & Material Moving,Occ%:Business & Financial,"Occ%:Computer, Mathematical, & Engineering","Occ%:Education, Legal, Social, Arts, & Media","Occ%:Food, Property, & Personal Services",Occ%:Healthcare Practitioners & Technical,Occ%:Healthcare Support,"Occ%:Natural Resources, Construction, & Maintenance",Occ%:Production,Occ%:Protective Services,Occ%:Sales & Office Support,Occ%:Total,Occ%:Transportation & Material Moving
0,"Adams city, Tennessee",2014,4.121418,0.583411,5.734492,12.071051,0.919385,3.697354,3.248877,0.684419,1.054255,18.95524,58.199974,7.130073,7.081477,1.002424,9.853084,20.740646,1.5797,6.352844,5.582266,1.175978,1.811436,32.569154,NaN,12.25099
1,"Adams city, Tennessee",2015,4.895225,0.678645,6.06584,17.487364,1.471388,3.902959,4.720314,0.704735,1.541629,17.805229,65.864417,6.59109,7.432275,1.030367,9.209586,26.550549,2.233965,5.925748,7.166713,1.069978,2.34061,27.033153,NaN,10.007057


In [167]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [168]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Occ%:Business & Financial                                                                                    \
Year                                              2014       2015      2016      2017      2018      2019      2020      2021       2022   
NAME                                                                                                                                       
Adams city, Tennessee                         7.081477   7.432275  7.866789  8.516995  8.964910  8.869543  9.071620  9.972795  10.757196   
Ashland City town, Tennessee                  9.458728  10.348177  9.748299  9.060580  9.209491  9.415043  9.562034  9.942892  10.330264   

                             Occ%:Computer, Mathematical, & Engineering                                                                        \
Year                                                               2014      2015      2016      2017      2018      2019      2020      2021   
NAME                                                                                                                                            
Adams city, Tennessee                                          1.002424  1.030367  1.301811  1.361399  1.209732  1.125928  1.138289  1.181191   
Ashland City town, Tennessee                                   4.298313  4.546314  4.498096  4.320272  4.453209  4.641814  4.867281  5.004130   

                                       Occ%:Education, Legal, Social, Arts, & Media                                                              \
Year                              2022                                         2014      2015      2016      2017      2018      2019      2020   
NAME                                                                                                                                              
Adams city, Tennessee         1.233159                                     9.853084  9.209586  9.006654  9.315401  9.207809  8.346916  6.892152   
Ashland City town, Tennessee  5.107768                                     4.471206  4.530169  4.330997  4.256910  4.596766  4.513836  4.250190   

                                                 Occ%:Food, Property, & Personal Services                                                         \
Year                              2021      2022                                     2014       2015       2016       2017       2018       2019   
NAME                                                                                                                                               
Adams city, Tennessee         6.100527  6.135656                                20.740646  26.550549  23.252644  19.238139  19.263678  19.562909   
Ashland City town, Tennessee  3.999086  3.866837                                10.892360  10.590088  10.676158  11.204989  11.253573  10.932904   

                                                              Occ%:Healthcare Practitioners & Technical                                          \
Year                               2020       2021       2022                                      2014      2015      2016      2017      2018   
NAME                                                                                                                                              
Adams city, Tennessee         19.674252  15.732578  14.211417                                  1.579700  2.233965  3.400304  4.018037  4.400261   
Ashland City town, Tennessee   9.926953   9.425118   9.423383                                  3.005015  2.920270  2.964305  2.802435  2.733549   

                                                                     Occ%:Healthcare Support                                                    \
Year                              2019      2020      2021      2022                    2014      2015      2016      2017      2018      2019   
NAME                                                                                             

In [169]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Occ%:Business & Financial                                                                                     \
                                                  2014       2015      2016      2017      2018      2019      2020       2021       2022   
                                                  None       None      None      None      None      None      None       None       None   
NAME                                                                                                                                        
Adams city, Tennessee                         7.081477   7.432275  7.866789  8.516995  8.964910  8.869543  9.071620   9.972795  10.757196   
Ashland City town, Tennessee                  9.458728  10.348177  9.748299  9.060580  9.209491  9.415043  9.562034   9.942892  10.330264   
Belle Meade city, Tennessee                   7.880750   7.871978  7.902902  8.284656  8.648276  8.815233  9.453686  10.065401  10.404400   

                             Occ%:Computer, Mathematical, & Engineering                                                                        \
                                                                   2014      2015      2016      2017      2018      2019      2020      2021   
                                                                   None      None      None      None      None      None      None      None   
NAME                                                                                                                                            
Adams city, Tennessee                                          1.002424  1.030367  1.301811  1.361399  1.209732  1.125928  1.138289  1.181191   
Ashland City town, Tennessee                                   4.298313  4.546314  4.498096  4.320272  4.453209  4.641814  4.867281  5.004130   
Belle Meade city, Tennessee                                    1.470794  1.269278  1.183948  1.238654  1.286920  1.296613  1.670886  1.991710   

                                       Occ%:Education, Legal, Social, Arts, & Media                                                                 \
                                  2022                                         2014       2015       2016       2017      2018      2019      2020   
                                  None                                         None       None       None       None      None      None      None   
NAME                                                                                                                                                 
Adams city, Tennessee         1.233159                                     9.853084   9.209586   9.006654   9.315401  9.207809  8.346916  6.892152   
Ashland City town, Tennessee  5.107768                                     4.471206   4.530169   4.330997   4.256910  4.596766  4.513836  4.250190   
Belle Meade city, Tennessee   1.984569                                    12.667565  11.746149  11.469898  10.686087  9.560125  9.049961  8.943649   

                                                 Occ%:Food, Property, & Personal Services                                                         \
                                  2021      2022                                     2014       2015       2016       2017       2018       2019   
                                  None      None                                     None       None       None       None       None       None   
NAME                                                                                                                                               
Adams city, Tennessee         6.100527  6.135656                                20.740646  26.550549  23.252644  19.238139  19.263678  19.562909   
Ashland City town, Tennessee  3.999086  3.866837                                10.892360  10.590088  10.676158  11.204989  11.253573  10.932904   
Belle Meade city, Tennessee   8.831333  8.623603                                39.619

In [170]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = data.columns.get_level_values(0).unique().tolist()
first_level.remove('NAME')
first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [171]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [172]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [173]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [174]:
#final check
data.head(3)

,NAME,Year,Time Frame,Occ%:Business & Financial,"Occ%:Computer, Mathematical, & Engineering","Occ%:Education, Legal, Social, Arts, & Media","Occ%:Food, Property, & Personal Services",Occ%:Healthcare Practitioners & Technical,Occ%:Healthcare Support,"Occ%:Natural Resources, Construction, & Maintenance",Occ%:Production,Occ%:Protective Services,Occ%:Sales & Office Support,Occ%:Transportation & Material Moving,Occ:Business & Financial,Occ:Business & Financial % Change,Occ:Business & Financial Change,"Occ:Computer, Mathematical, & Engineering","Occ:Computer, Mathematical, & Engineering % Change","Occ:Computer, Mathematical, & Engineering Change","Occ:Education, Legal, Social, Arts, & Media","Occ:Education, Legal, Social, Arts, & Media % Change","Occ:Education, Legal, Social, Arts, & Media Change","Occ:Food, Property, & Personal Services","Occ:Food, Property, & Personal Services % Change","Occ:Food, Property, & Personal Services Change",Occ:Healthcare Practitioners & Technical,Occ:Healthcare Practitioners & Technical % Change,Occ:Healthcare Practitioners & Technical Change,Occ:Healthcare Support,Occ:Healthcare Support % Change,Occ:Healthcare Support Change,"Occ:Natural Resources, Construction, & Maintenance","Occ:Natural Resources, Construction, & Maintenance % Change","Occ:Natural Resources, Construction, & Maintenance Change",Occ:Production,Occ:Production % Change,Occ:Production Change,Occ:Protective Services,Occ:Protective Services % Change,Occ:Protective Services Change,Occ:Sales & Office Support,Occ:Sales & Office Support % Change,Occ:Sales & Office Support Change,Occ:Total,Occ:Total % Change,Occ:Total Change,Occ:Transportation & Material Moving,Occ:Transportation & Material Moving % Change,Occ:Transportation & Material Moving Change,GEO_ID,Source
0,"Adams city, Tennessee",2014,None,7.081477,1.002424,9.853084,20.740646,1.579700,6.352844,5.582266,1.175978,1.811436,32.569154,12.250990,4.121418,NaN,NaN,0.583411,NaN,NaN,5.734492,NaN,NaN,12.071051,NaN,NaN,0.919385,NaN,NaN,3.697354,NaN,NaN,3.248877,NaN,NaN,0.684419,NaN,NaN,1.054255,NaN,NaN,18.955240,NaN,NaN,58.199974,NaN,NaN,7.130073,NaN,NaN,1600000US4700200,JobsEQ
1,"Adams city, Tennessee",2015,None,7.432275,1.030367,9.209586,26.550549,2.233965,5.925748,7.166713,1.069978,2.340610,27.033153,10.007057,4.895225,NaN,NaN,0.678645,NaN,NaN,6.065840,NaN,NaN,17.487364,NaN,NaN,1.471388,NaN,NaN,3.902959,NaN,NaN,4.720314,NaN,NaN,0.704735,NaN,NaN,1.541629,NaN,NaN,17.805229,NaN,NaN,65.864417,NaN,NaN,6.591090,NaN,NaN,1600000US4700200,JobsEQ
2,"Adams city, Tennessee",2016,None,7.866789,1.301811,9.006654,23.252644,3.400304,5.515605,7.322429,1.029413,4.026679,28.366467,8.911205,5.281387,NaN,NaN,0.873974,NaN,NaN,6.046638,NaN,NaN,15.610716,NaN,NaN,2.282802,NaN,NaN,3.702914,NaN,NaN,4.915930,NaN,NaN,0.691099,NaN,NaN,2.703320,NaN,NaN,19.043892,NaN,NaN,67.135229,NaN,NaN,5.982558,NaN,NaN,1600000US4700200,JobsEQ


In [175]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3870 entries, 0 to 3869
Data columns (total 52 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   NAME                                                         3870 non-null   object 
 1   Year                                                         3870 non-null   object 
 2   Time Frame                                                   3870 non-null   object 
 3   Occ%:Business & Financial                                    774 non-null    float64
 4   Occ%:Computer, Mathematical, & Engineering                   774 non-null    float64
 5   Occ%:Education, Legal, Social, Arts, & Media                 774 non-null    float64
 6   Occ%:Food, Property, & Personal Services                     774 non-null    float64
 7   Occ%:Healthcare Practitioners & Technical                    774 non-null    f

In [69]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_OccupationalEmployment_Annual_Change', conn, if_exists = 'replace', index = False)

3870

The Woods & Poole data that is used more at the county level is only used at the place level for total employment projections. This is for consistency. This is a land use model ultimately, so JobsEQ employment is selected to model place level employment for place of work. This data is derived from the QCEW and other inputs so is selected as a better option - both are modeled so we use the one modeled directly from UI claims rather than land use at this granular of a geography.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function 
+ *load the following query: JEQAnnual_Industry**

The guidance for formatting this query manually is below:  
+ Select the "Industry" dataset, annual  
+ Select the following dimensions: Region, Industry, Year/Quarter, then select "Employment" as the value  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Census Places"   
+ For "Year/Quarter" select as many as possible  
+ Under "Industry" expand "Total-All Industries" and select the combination of 2 digit NAICS codes and built groups in Jobs EQ as described below, leaving Total as well  

Industries to Select:  
+ Total - All Industries  
(Custom Created Groups *begin to type them as they appear here and they will appear in the search bar, if not then remake the custom group as described below*)  
+ Snapshot 11: Construction (23: Construction)  
+ Snapshot 11: Manufacturing (31: Manufacturing)  
+ Snapshot 11: Information (51: Information)  
+ Snapshot 11: Public Administration (92: Public Administration)  
+ Snapshot 11: Other/Unclassified (81: Other, Except Public Administration, 99: Unclassified)  
+ Snapshot 11: Leisure & Hospitality (71: Arts, Entertainment, & Recreation, 72: Accommodation & Food Services)   
+ Snapshot 11: Education & Health Services (61: Education, 62: Healthcare & Social Assistance)   
+ Snapshot 11: Professional & Business Services (54: Professional, Scientific, & Technical, 55: Management of Companies & Enterprises, 56: Administrative & Support & Waste Management & Remediation)   
+ Snapshot 11: Financial Activities (52: Finance & Insurance, 53: Real Estate & Rental & Leasing)   
+ Snapshot 11: Trade, Transportation & Utilities (42: Wholesale Trade, 44-45: Retail Trade, 48-49: Transportation & Warehousing, 22: Utilities)  
+ Snapshot 11: Natural Resources & Mining (11: Agricultural Activities, Forestry, Fishing, & Hunting, 21: Mining, Quarrying, Oil & Gas) 
+ Snapshot: White Collar  
+ Snapshot: Blue Collar  
+ Snapshot 4: Government  
+ Snapshot 4: Retail & Hospitality  
+ Snapshot 4: Professional Services  
+ Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics  

Select: 'Employment'

This may exceed the drafting size, in which case you'll have to queue for download. Make sure Industry is on the top and Region and Time are on the left side. 

*After loading the query and downloading, resume here*  

Once it is completed, open and delete the top row that says "Industry Data", and delete the third row that repeats "Empl". Delete the "FIPS" column, change "Time" to "Year", and "Region" to "NAME". Select all numerical data and format to "general". Delete metadata at the bottom. Save in the "Data Downloads" folder as "JobsEQ_IndustryEmploymentAnnual_AllGeos.csv"

In [78]:
#import and examine, dtype str so the GEO_ID formatting doesn't get messed up and so that we can split the time column
data = pd.read_csv('../../Data Downloads/JobsEQ_IndustryEmploymentAnnual_AllGeos.csv', dtype = str)
data.head(3)

,NAME,Year,Total - All Industries,Snapshot: Blue Collar,Snapshot: White Collar,Snapshot 11: Information,Snapshot 11: Construction,Snapshot 11: Manufacturing,Snapshot 11: Public Administration,Snapshot 11: Other/Unclassified,Snapshot 11: Leisure & Hospitality,Snapshot 11: Education & Health Services,Snapshot 11: Professional & Business Services,Snapshot 11: Financial Activities,"Snapshot 11: Trade, Transportation, & Utilities",Snapshot 11: Natural Resources & Mining,Snapshot 4: Government,Snapshot 4: Retail & Hospitality,Snapshot 4: Professional Services,"Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics"
0,USA,2001,141306325.3,38834471.94,55852069.3,3878268.435,8912180.533,16739169.03,7041142.738,6855080.969,12834185.56,26555121.25,18317213.8,8600855.94,28715784.21,2857322.859,7041142.738,26428519.67,66698649.9,41138013.01
1,USA,2002,139803504.2,37178089.33,56176710.67,3651345.192,8780878.023,15557016.02,7137276.32,6918331.804,12947416.73,27254062.38,17898061.36,8612536.813,28221663.98,2824915.527,7137276.32,26343612.61,66843777.74,39478837.49
2,USA,2003,139767237.5,36416782.29,56690864.85,3473712.439,8857288.326,14822296.98,7147347.083,7011711.302,13158602.46,27785501.39,17891963.68,8770941.01,28073059.98,2774812.901,7147347.083,26461911.13,67485556.43,38672422.91


In [79]:
#quickly rename to full census geographies
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [80]:
#make a list of years to loop through grouping incorporated and unincorporated areas
times = list(data['Year'].unique())

In [81]:
data = data.set_index(['NAME', 'Year']).transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [82]:
#aggregate by double column-headers
for time in times:
    RuthInc = [data[('Eagleville city, Tennessee', '{}'.format(time))],data[('La Vergne city, Tennessee', '{}'.format(time))],
               data[('Murfreesboro city, Tennessee', '{}'.format(time))],data[('Smyrna town, Tennessee', '{}'.format(time))]]
    data[('Rutherford Incorporated', '{}'.format(time))] = sum(RuthInc)
    data[('Rutherford Unincorporated', '{}'.format(time))] = data[('Rutherford County, Tennessee', '{}'.format(time))] - data[('Rutherford Incorporated', '{}'.format(time))]
    WilsonInc = [data[('Lebanon city, Tennessee', '{}'.format(time))],data[('Mount Juliet city, Tennessee', '{}'.format(time))],
                 data[('Watertown city, Tennessee', '{}'.format(time))]]
    data[('Wilson Incorporated', '{}'.format(time))] = sum(WilsonInc)
    data[('Wilson Unincorporated', '{}'.format(time))] = data[('Wilson County, Tennessee', '{}'.format(time))] - data[('Wilson Incorporated', '{}'.format(time))]
    CheathInc = [data[('Ashland City town, Tennessee', '{}'.format(time))],data[('Kingston Springs town, Tennessee', '{}'.format(time))],
                 data[('Pegram town, Tennessee', '{}'.format(time))],data[('Pleasant View city, Tennessee', '{}'.format(time))]]
    data[('Cheatham Incorporated', '{}'.format(time))] = sum(CheathInc)
    data[('Cheatham Unincorporated', '{}'.format(time))] = data[('Cheatham County, Tennessee', '{}'.format(time))] - data[('Cheatham Incorporated', '{}'.format(time))]
    DicksInc = [data[('Burns town, Tennessee', '{}'.format(time))],data[('Charlotte town, Tennessee', '{}'.format(time))],
                data[('Dickson city, Tennessee', '{}'.format(time))],data[('Slayden town, Tennessee', '{}'.format(time))],
                data[('Vanleer town, Tennessee', '{}'.format(time))],data[('White Bluff town, Tennessee', '{}'.format(time))]]
    data[('Dickson Incorporated', '{}'.format(time))] = sum(DicksInc)
    data[('Dickson Unincorporated', '{}'.format(time))] = data[('Dickson County, Tennessee', '{}'.format(time))] - data[('Dickson Incorporated', '{}'.format(time))]
    HumphInc = [data[('McEwen city, Tennessee', '{}'.format(time))],data[('New Johnsonville city, Tennessee', '{}'.format(time))],
                data[('Waverly city, Tennessee', '{}'.format(time))]]
    data[('Humphreys Incorporated', '{}'.format(time))] = sum(HumphInc)
    data[('Humphreys Unincorporated', '{}'.format(time))] = data[('Humphreys County, Tennessee', '{}'.format(time))] - data[('Humphreys Incorporated', '{}'.format(time))]
    data[('Montgomery Incorporated', '{}'.format(time))] = data[('Clarksville city, Tennessee', '{}'.format(time))]
    data[('Montgomery Unincorporated', '{}'.format(time))] = data[('Montgomery County, Tennessee', '{}'.format(time))] - data[('Montgomery Incorporated', '{}'.format(time))]

In [83]:
data = data.transpose()
data.head()

Total - All Industries  Snapshot: Blue Collar  Snapshot: White Collar  Snapshot 11: Information  Snapshot 11: Construction  \
NAME          Year                                                                                                                               
United States 2001             141306325.3            38834471.94             55852069.30               3878268.435                8912180.533   
              2002             139803504.2            37178089.33             56176710.67               3651345.192                8780878.023   
              2003             139767237.5            36416782.29             56690864.85               3473712.439                8857288.326   
              2004             141407560.6            36595037.76             57298446.43               3395314.312                9124554.627   
              2005             143710293.3            37115842.35             58354337.83               3352726.674                9514656.814   

                    Snapshot 11: Manufacturing  Snapshot 11: Public Administration  Snapshot 11: Other/Unclassified  \
NAME          Year                                                                                                    
United States 2001                 16739169.03                         7041142.738                      6855080.969   
              2002                 15557016.02                         7137276.320                      6918331.804   
              2003                 14822296.98                         7147347.083                      7011711.302   
              2004                 14613043.93                         7118854.350                      7102385.265   
              2005                 14528052.80                         7149279.661                      7124029.660   

                    Snapshot 11: Leisure & Hospitality  Snapshot 11: Education & Health Services  Snapshot 11: Professional & Business Services  \
NAME          Year                                                                                                                                
United States 2001                         12834185.56                               26555121.25                                    18317213.80   
              2002                         12947416.73                               27254062.38                                    17898061.36   
              2003                         13158602.46                               27785501.39                                    17891963.68   
              2004                         13486787.31                               28234240.74                                    18365801.16   
              2005                         13774781.69                               28774741.87                                    18969862.45   

                    Snapshot 11: Financial Activities  Snapshot 11: Trade, Transportation, & Utilities  Snapshot 11: Natural Resources & Mining  \
NAME          Year                                                                                                                                
United States 2001                        8600855.940                                      28715784.21                              2857322.859   
              2002                        8612536.813                                      28221663.98                              2824915.527   
              2003                        8770941.010                                      28073059.98                              2774812.901   
              2004                        8844604.534                                      28313276.95                              2808697.407   
              2005                        9030081.874                                      28646637.00                              2845442.792   

                    Snapshot 4: Government  Snapshot 4: Retail & Hospitality  Snapshot 4: Professional 

In [84]:
#sum these, note that this categorization for service producing doesn't contain trade, transportation & utilities as that's broken out separately
thelist = [data['Snapshot 11: Natural Resources & Mining'], data['Snapshot 11: Construction'], data['Snapshot 11: Manufacturing']]
data['Goods Producing'] = sum(thelist)
thelist = [data['Snapshot 11: Information'], data['Snapshot 11: Financial Activities'], data['Snapshot 11: Professional & Business Services'], 
           data['Snapshot 11: Education & Health Services'], data['Snapshot 11: Leisure & Hospitality'], data['Snapshot 11: Public Administration'], 
           data['Snapshot 11: Other/Unclassified']]
data['Service Producing'] = sum(thelist)

In [85]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['Snapshot 11: Construction'], data['Snapshot 11: Manufacturing'], 
           data['Snapshot 11: Information'], data['Snapshot 11: Public Administration'], 
           data['Snapshot 11: Other/Unclassified'], data['Snapshot 11: Leisure & Hospitality'],
           data['Snapshot 11: Education & Health Services'], data['Snapshot 11: Professional & Business Services'], 
           data['Snapshot 11: Financial Activities'], data['Snapshot 11: Trade, Transportation, & Utilities'], data['Snapshot 11: Natural Resources & Mining']]
data['Total - All Industries'] = sum(thelist)

In [86]:
#rename columns to account for the disparity between custom groups and NAICS 2 digit formatting
data = data.rename(columns = {'Total - All Industries': 'Total', 'Snapshot 11: Construction': 'Construction',
                              'Snapshot 11: Manufacturing': 'Manufacturing', 'Snapshot 11: Information': 'Information', 
                              'Snapshot 11: Public Administration': 'Public Administration', 
                              'Snapshot 11: Other/Unclassified': 'Other/Unclassified', 
                              'Snapshot 11: Leisure & Hospitality': 'Leisure & Hospitality', 
                              'Snapshot 11: Education & Health Services': 'Education & Health Services', 
                              'Snapshot 11: Professional & Business Services': 'Professional & Business Services', 
                              'Snapshot 11: Financial Activities': 'Financial Activities', 
                              'Snapshot 11: Trade, Transportation, & Utilities': 'Trade, Transportation, & Utilities', 
                              'Snapshot 11: Natural Resources & Mining': 'Natural Resources & Mining'
                             })

In [87]:
#set the index to year and name so that we can add a prefix to all of the other columns, before resetting the index without dropping
data.reset_index(drop = False, inplace = True)
data = data.set_index(['NAME', 'Year']).add_prefix('Ind:').reset_index(drop = False)

In [88]:
#melt this data down using the columns name and year to distinguish specific instances, and then attributing total employment per occupation
#name it separately because it bugs out if you call this data and merge later
cols = data.columns
thing = data.melt(var_name = 'Industry', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Employment')
thing.head(2)

,NAME,Year,Industry,Employment
0,United States,2001,Ind:Total,1.413063e+08
1,United States,2002,Ind:Total,1.398035e+08


In [89]:
#create a separate dataframe where we will calculate percent of the total for each occupation group
percents = data

In [90]:
#calculate percent of total
percents['Ind:Construction'] = percent(percents['Ind:Construction'], percents['Ind:Total'])
percents['Ind:Manufacturing'] = percent(percents['Ind:Manufacturing'], percents['Ind:Total'])
percents['Ind:Information'] = percent(percents['Ind:Information'], percents['Ind:Total'])
percents['Ind:Public Administration'] = percent(percents['Ind:Public Administration'], percents['Ind:Total'])
percents['Ind:Other/Unclassified'] = percent(percents['Ind:Other/Unclassified'], percents['Ind:Total'])
percents['Ind:Leisure & Hospitality'] = percent(percents['Ind:Leisure & Hospitality'], percents['Ind:Total'])
percents['Ind:Education & Health Services'] = percent(percents['Ind:Education & Health Services'], percents['Ind:Total'])
percents['Ind:Professional & Business Services'] = percent(percents['Ind:Professional & Business Services'], percents['Ind:Total'])
percents['Ind:Financial Activities'] = percent(percents['Ind:Financial Activities'], percents['Ind:Total'])
percents['Ind:Trade, Transportation, & Utilities'] = percent(percents['Ind:Trade, Transportation, & Utilities'], percents['Ind:Total'])
percents['Ind:Natural Resources & Mining'] = percent(percents['Ind:Natural Resources & Mining'], percents['Ind:Total'])
percents['Ind:Goods Producing'] = percent(percents['Ind:Goods Producing'], percents['Ind:Total'])
percents['Ind:Service Producing'] = percent(percents['Ind:Service Producing'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Government'] = percent(percents['Ind:Snapshot 4: Government'], percents['Ind:Total'])
percents['Ind:Snapshot: Blue Collar'] = percent(percents['Ind:Snapshot: Blue Collar'], percents['Ind:Total'])
percents['Ind:Snapshot: White Collar'] = percent(percents['Ind:Snapshot: White Collar'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Retail & Hospitality'] = percent(percents['Ind:Snapshot 4: Retail & Hospitality'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Professional Services'] = percent(percents['Ind:Snapshot 4: Professional Services'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'] = percent(percents['Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'], 
                                                                                      percents['Ind:Total'])

In [91]:
#similarly melt this dataframe
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Industry', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Percent of Employment')
percents.head()

,NAME,Year,Industry,Percent of Employment
0,United States,2001,Ind:Total,1.413063e+08
1,United States,2002,Ind:Total,1.398035e+08
2,United States,2003,Ind:Total,1.397672e+08
3,United States,2004,Ind:Total,1.414076e+08
4,United States,2005,Ind:Total,1.437103e+08


In [92]:
#we never renamed the total column when we duplicated the dataframe for percents, we want this to be null so just transpose, drop it, and transpose back
percents = percents.set_index('Industry').transpose()
percents = percents.drop(columns = 'Ind:Total')
percents = percents.transpose().reset_index(drop = False)

In [93]:
#merge total employment and percent of total employment
data = thing.merge(percents, on = ['Industry', 'Year', 'NAME'], how = 'outer')
data.head(3)

,NAME,Year,Industry,Employment,Percent of Employment
0,United States,2001,Ind:Total,1.413063e+08,NaN
1,United States,2002,Ind:Total,1.398035e+08,NaN
2,United States,2003,Ind:Total,1.397672e+08,NaN


In [94]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos beforeresetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose().reset_index(drop = False)

In [95]:
#set the index to "index" (the variable names) and transpose, then reset the index without dropping it
data = data.set_index('index').transpose().reset_index(drop = False)

In [96]:
data.head()

index,NAME,Year,Industry,Employment,Percent of Employment
0,United States,2001,Ind:Total,141306325.324,NaN
1,United States,2002,Ind:Total,139803504.149,NaN
2,United States,2003,Ind:Total,139767237.551,NaN
3,United States,2004,Ind:Total,141407560.585,NaN
4,United States,2005,Ind:Total,143710293.285,NaN


In [97]:
#go long to wide
data = pd.pivot(data, index = ['NAME', 'Year'], columns = 'Industry', values = ['Employment', 'Percent of Employment'])

In [98]:
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
data.columns = data.columns.to_flat_index()

In [99]:
#rename
inddict = {('Employment', 'Ind:Education & Health Services'): 'Ind:Education & Health Services', 
           ('Employment', 'Ind:Construction'): 'Ind:Construction', 
           ('Employment', 'Ind:Financial Activities'): 'Ind:Financial Activities',
           ('Employment', 'Ind:Information'): 'Ind:Information', 
           ('Employment', 'Ind:Manufacturing'): 'Ind:Manufacturing', 
           ('Employment', 'Ind:Other/Unclassified'): 'Ind:Other/Unclassified', 
           ('Employment', 'Ind:Public Administration'): 'Ind:Public Administration', 
           ('Employment', 'Ind:Total'): 'Ind:Total', 
           ('Employment', 'Ind:Goods Producing'): 'Ind:Goods Producing', 
           ('Employment', 'Ind:Leisure & Hospitality'): 'Ind:Leisure & Hospitality', 
           ('Employment', 'Ind:Natural Resources & Mining'): 'Ind:Natural Resources & Mining', 
           ('Employment', 'Ind:Professional & Business Services'): 'Ind:Professional & Business Services', 
           ('Employment', 'Ind:Service Producing'): 'Ind:Service Producing', 
           ('Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind:Trade, Transportation, & Utilities',
           ('Employment', 'Ind:Snapshot 4: Government'): 'Ind:Snapshot: Government',
           ('Employment', 'Ind:Snapshot: Blue Collar'): 'Ind:Snapshot: Blue Collar',
           ('Employment', 'Ind:Snapshot: White Collar'): 'Ind:Snapshot: White Collar',
           ('Employment', 'Ind:Snapshot 4: Retail & Hospitality'): 'Ind:Snapshot: Retail & Hospitality',
           ('Employment', 'Ind:Snapshot 4: Professional Services'): 'Ind:Snapshot: Professional Services',
           ('Employment', 'Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics',
           ('Percent of Employment', 'Ind:Education & Health Services'): 'Ind%:Education & Health Services',
           ('Percent of Employment', 'Ind:Construction'): 'Ind%:Construction', 
           ('Percent of Employment', 'Ind:Financial Activities'): 'Ind%:Financial Activities',
           ('Percent of Employment', 'Ind:Information'): 'Ind%:Information', 
           ('Percent of Employment', 'Ind:Manufacturing'): 'Ind%:Manufacturing', 
           ('Percent of Employment', 'Ind:Other/Unclassified'): 'Ind%:Other/Unclassified', 
           ('Percent of Employment', 'Ind:Public Administration'): 'Ind%:Public Administration', 
           ('Percent of Employment', 'Ind:Total'): 'Ind%:Total', 
           ('Percent of Employment', 'Ind:Goods Producing'): 'Ind%:Goods Producing', 
           ('Percent of Employment', 'Ind:Leisure & Hospitality'): 'Ind%:Leisure & Hospitality', 
           ('Percent of Employment', 'Ind:Natural Resources & Mining'): 'Ind%:Natural Resources & Mining', 
           ('Percent of Employment', 'Ind:Professional & Business Services'): 'Ind%:Professional & Business Services', 
           ('Percent of Employment', 'Ind:Service Producing'): 'Ind%:Service Producing', 
           ('Percent of Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind%:Trade, Transportation, & Utilities', 
           ('Percent of Employment', 'Ind:Snapshot 4: Government'): 'Ind%:Snapshot: Government',
           ('Percent of Employment', 'Ind:Snapshot: Blue Collar'): 'Ind%:Snapshot: Blue Collar',
           ('Percent of Employment', 'Ind:Snapshot: White Collar'): 'Ind%:Snapshot: White Collar',
           ('Percent of Employment', 'Ind:Snapshot 4: Retail & Hospitality'): 'Ind%:Snapshot: Retail & Hospitality',
           ('Percent of Employment', 'Ind:Snapshot 4: Professional Services'): 'Ind%:Snapshot: Professional Services',
           ('Percent of Employment', 'Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'
           }
data = data.rename(columns = inddict)
data = data.reset_index(drop = False)

In [100]:
#make sure we have full census names
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [101]:
data.head(2)

,NAME,Year,Ind:Construction,Ind:Education & Health Services,Ind:Financial Activities,Ind:Goods Producing,Ind:Information,Ind:Leisure & Hospitality,Ind:Manufacturing,Ind:Natural Resources & Mining,Ind:Other/Unclassified,Ind:Professional & Business Services,Ind:Public Administration,Ind:Service Producing,Ind:Snapshot: Government,"Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind:Snapshot: Professional Services,Ind:Snapshot: Retail & Hospitality,Ind:Snapshot: Blue Collar,Ind:Snapshot: White Collar,Ind:Total,"Ind:Trade, Transportation, & Utilities",Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot: Government,"Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot: Professional Services,Ind%:Snapshot: Retail & Hospitality,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: White Collar,Ind%:Total,"Ind%:Trade, Transportation, & Utilities"
0,"Adams city, Tennessee",2001,0.38355,0.216182,0.199042,1.782378,0.000755,0.164992,0.001935,1.396893,2.105875,0.554439,15.649737,18.891022,15.649737,12.354024,3.241153,2.403308,10.95713,16.530456,33.648222,12.974821,1.139882,0.642475,0.591538,5.297094,0.002243,0.490345,0.00575,4.151462,6.258503,1.647751,46.50985,56.142706,46.50985,36.715235,9.632465,7.14245,32.563773,49.127279,NaN,38.5602
1,"Adams city, Tennessee",2002,0.372381,0.211523,0.20454,1.759003,0.001706,0.161598,0.003036,1.383587,1.770112,0.540403,16.764184,19.654066,16.764184,11.116233,2.889753,2.474226,9.73272,17.634467,33.244394,11.831326,1.120131,0.636267,0.615263,5.291127,0.005131,0.486092,0.009131,4.161866,5.324542,1.625547,50.427098,59.119939,50.427098,33.437916,8.692451,7.442534,29.276273,53.044933,NaN,35.588934


In [102]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [103]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Ind%:Construction                                                                                                       \
Year                                      2001      2002      2003      2004      2005       2006      2007      2008      2009      2010      2011   
NAME                                                                                                                                                  
Adams city, Tennessee                 1.139882  1.120131  0.928107  0.779861  0.552338  10.339906  0.420782  0.355615  0.413163  7.216010  6.178453   
Ashland City town, Tennessee          6.564724  5.263736  3.831064  3.564937  3.841972   4.194350  2.435508  2.409556  2.760593  2.633429  2.609661   

                                                                                                                                            \
Year                              2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
NAME                                                                                                                                         
Adams city, Tennessee         0.446641  2.915054  0.348886  6.054197  2.351477  2.202285  2.654073  0.517368  3.317016  3.412854  3.459101   
Ashland City town, Tennessee  3.042124  3.355103  3.607253  2.091791  1.872051  1.994972  2.055946  3.317894  2.407220  2.415015  2.639809   

                             Ind%:Education & Health Services                                                                                  \
Year                                                     2001      2002      2003      2004      2005      2006      2007      2008      2009   
NAME                                                                                                                                            
Adams city, Tennessee                                0.642475  0.636267  0.515166  0.433192  0.325713  0.443120  0.507097  0.238349  0.359815   
Ashland City town, Tennessee                        10.858217  8.703648  8.086868  8.032417  8.543300  7.165599  7.918047  7.414795  7.996406   

                                                                                                                                                     \
Year                              2010       2011       2012       2013       2014       2015       2016       2017       2018       2019      2020   
NAME                                                                                                                                                  
Adams city, Tennessee         0.760582  19.895576  24.759166  27.924843  25.460818  23.034189  22.795114  24.901019  25.720949  23.742303  6.279033   
Ashland City town, Tennessee  8.744628   7.897518   8.725147   9.029418   9.952656   8.723468   8.579080   8.617972   7.962309   7.906226  7.755924   

                                                 Ind%:Financial Activities                                                                          \
Year                              2021      2022                      2001      2002      2003      2004       2005      2006       2007      2008   
NAME                                                                                                                                                 
Adams city, Tennessee         6.828690  6.548316                  0.591538  0.615263  1.751128  0.330795  21.603882  6.916420  11.741138  7.831860   
Ashland City town, Tennessee  7.819367  7.785223                  3.916684  3.333328  3.762979  3.555864   3.148324  3.225533   3.654649  3.663823   

                                                                                                                                            \
Year                              2009      2010      2011      2012      2013      2014      2015      2016      2017      2018      2019   
NAME                                         

In [104]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Ind%:Construction                                                                                                       \
                                          2001      2002      2003      2004      2005       2006      2007      2008      2009      2010      2011   
                                          None      None      None      None      None       None      None      None      None      None      None   
NAME                                                                                                                                                  
Adams city, Tennessee                 1.139882  1.120131  0.928107  0.779861  0.552338  10.339906  0.420782  0.355615  0.413163  7.216010  6.178453   
Ashland City town, Tennessee          6.564724  5.263736  3.831064  3.564937  3.841972   4.194350  2.435508  2.409556  2.760593  2.633429  2.609661   
Belle Meade city, Tennessee           1.070429  1.005908  1.523918  1.072889  1.003974   2.642801  1.421578  1.529968  1.240862  1.636436  3.002117   

                                                                                                                                            \
                                  2012      2013      2014      2015      2016      2017      2018      2019      2020      2021      2022   
                                  None      None      None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                         
Adams city, Tennessee         0.446641  2.915054  0.348886  6.054197  2.351477  2.202285  2.654073  0.517368  3.317016  3.412854  3.459101   
Ashland City town, Tennessee  3.042124  3.355103  3.607253  2.091791  1.872051  1.994972  2.055946  3.317894  2.407220  2.415015  2.639809   
Belle Meade city, Tennessee   1.109671  1.052163  1.426193  2.850584  1.514652  1.882281  1.789059  1.728388  2.473149  2.435572  2.304578   

                             Ind%:Education & Health Services                                                                                  \
                                                         2001      2002      2003      2004      2005      2006      2007      2008      2009   
                                                         None      None      None      None      None      None      None      None      None   
NAME                                                                                                                                            
Adams city, Tennessee                                0.642475  0.636267  0.515166  0.433192  0.325713  0.443120  0.507097  0.238349  0.359815   
Ashland City town, Tennessee                        10.858217  8.703648  8.086868  8.032417  8.543300  7.165599  7.918047  7.414795  7.996406   
Belle Meade city, Tennessee                          4.749518  4.998682  3.932546  4.808057  3.105113  3.213938  3.909409  3.679228  2.542554   

                                                                                                                                                     \
                                  2010       2011       2012       2013       2014       2015       2016       2017       2018       2019      2020   
                                  None       None       None       None       None       None       None       None       None       None      None   
NAME                                                                                                                                                  
Adams city, Tennessee         0.760582  19.895576  24.759166  27.924843  25.460818  23.034189  22.795114  24.901019  25.720949  23.742303  6.279033   
Ashland City town, Tennessee  8.744628   7.897518   8.725147   9.029418   9.952656   8.723468   8.579080   8.617972   7.962309   7.906226  7.755924   
Belle Meade city, Tennessee   3.281065   2.453717   4.

In [105]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
# first_level.remove('NAME')
# first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [106]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [107]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [108]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [109]:
#final check
data.head()

,NAME,Year,Time Frame,Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: Government,"Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot: Professional Services,Ind%:Snapshot: Retail & Hospitality,Ind%:Snapshot: White Collar,"Ind%:Trade, Transportation, & Utilities",Ind:Construction,Ind:Construction % Change,Ind:Construction Change,Ind:Education & Health Services,Ind:Education & Health Services % Change,Ind:Education & Health Services Change,Ind:Financial Activities,Ind:Financial Activities % Change,Ind:Financial Activities Change,Ind:Goods Producing,Ind:Goods Producing % Change,Ind:Goods Producing Change,Ind:Information,Ind:Information % Change,Ind:Information Change,Ind:Leisure & Hospitality,Ind:Leisure & Hospitality % Change,Ind:Leisure & Hospitality Change,Ind:Manufacturing,Ind:Manufacturing % Change,Ind:Manufacturing Change,Ind:Natural Resources & Mining,Ind:Natural Resources & Mining % Change,Ind:Natural Resources & Mining Change,Ind:Other/Unclassified,Ind:Other/Unclassified % Change,Ind:Other/Unclassified Change,Ind:Professional & Business Services,Ind:Professional & Business Services % Change,Ind:Professional & Business Services Change,Ind:Public Administration,Ind:Public Administration % Change,Ind:Public Administration Change,Ind:Service Producing,Ind:Service Producing % Change,Ind:Service Producing Change,Ind:Snapshot: Blue Collar,Ind:Snapshot: Blue Collar % Change,Ind:Snapshot: Blue Collar Change,Ind:Snapshot: Government,Ind:Snapshot: Government % Change,Ind:Snapshot: Government Change,"Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics % Change","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics Change",Ind:Snapshot: Professional Services,Ind:Snapshot: Professional Services % Change,Ind:Snapshot: Professional Services Change,Ind:Snapshot: Retail & Hospitality,Ind:Snapshot: Retail & Hospitality % Change,Ind:Snapshot: Retail & Hospitality Change,Ind:Snapshot: White Collar,Ind:Snapshot: White Collar % Change,Ind:Snapshot: White Collar Change,Ind:Total,Ind:Total % Change,Ind:Total Change,"Ind:Trade, Transportation, & Utilities","Ind:Trade, Transportation, & Utilities % Change","Ind:Trade, Transportation, & Utilities Change",GEO_ID,Source
0,"Adams city, Tennessee",2001,None,1.139882,0.642475,0.591538,5.297094,0.002243,0.490345,0.005750,4.151462,6.258503,1.647751,46.509850,56.142706,32.563773,46.509850,36.715235,9.632465,7.142450,49.127279,38.560200,0.383550,NaN,NaN,0.216182,NaN,NaN,0.199042,NaN,NaN,1.782378,NaN,NaN,0.000755,NaN,NaN,0.164992,NaN,NaN,0.001935,NaN,NaN,1.396893,NaN,NaN,2.105875,NaN,NaN,0.554439,NaN,NaN,15.649737,NaN,NaN,18.891022,NaN,NaN,10.957130,NaN,NaN,15.649737,NaN,NaN,12.354024,NaN,NaN,3.241153,NaN,NaN,2.403308,NaN,NaN,16.530456,NaN,NaN,33.648222,NaN,NaN,12.974821,NaN,NaN,1600000US4700200,JobsEQ
1,"Adams city, Tennessee",2002,None,1.120131,0.636267,0.615263,5.291127,0.005131,0.486092,0.009131,4.161866,5.324542,1.625547,50.427098,59.119939,29.276273,50.427098,33.437916,8.692451,7.442534,53.044933,35.588934,0.372381,NaN,NaN,0.211523,NaN,NaN,0.204540,NaN,NaN,1.759003,NaN,NaN,0.001706,NaN,NaN,0.161598,NaN,NaN,0.003036,NaN,NaN,1.383587,NaN,NaN,1.770112,NaN,NaN,0.540403,NaN,NaN,16.764184,NaN,NaN,19.654066,NaN,NaN,9.732720,NaN,NaN,16.764184,NaN,NaN,11.116233,NaN,NaN,2.889753,NaN,NaN,2.474226,NaN,NaN,17.634467,NaN,NaN,33.244394,NaN,NaN,11.831326,NaN,NaN,1600000US4700200,JobsEQ
2,"Adams city, Tennessee",2003,None,0.928107,0.515166,1.751128,3.904416,0.019976,3.474888,0.006807,2.969503,4.219358,1.335394,42.177830,53.493739,33.508641,42.177830,36.477971,8.234191,13.110008,45.585297,42.601845,0.399009,NaN,

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21758 entries, 0 to 21757
Data columns (total 84 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   NAME                                                                      21758 non-null  object 
 1   Year                                                                      21758 non-null  object 
 2   Time Frame                                                                21758 non-null  object 
 3   Ind%:Construction                                                         1892 non-null   float64
 4   Ind%:Education & Health Services                                          1892 non-null   float64
 5   Ind%:Financial Activities                                                 1892 non-null   float64
 6   Ind%:Goods Producing                                          

In [111]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_IndustryEmployment_Annual_Change', conn, if_exists = 'replace', index = False)

21758